### Ячейка 1: Импорты

In [184]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 1: Импорты и Базовая Настройка
# =============================================================================
print("--- Ячейка 1: Импорты и Базовая Настройка ---")

# --- Базовые библиотеки ---
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' # Для избежания конфликтов на некоторых системах
import zipfile
import time
import warnings
import json
import random
import traceback
from pathlib import Path
from typing import Union, Tuple, List, Dict, Optional

# --- Аудио и Сигналы ---
import librosa
import librosa.display
try:
    import torchaudio # Нужен для compute_deltas
    TORCHAUDIO_AVAILABLE = True
except ImportError:
    TORCHAUDIO_AVAILABLE = False
    print("!!! КРИТИЧЕСКАЯ ОШИБКА: torchaudio не найден, но необходим для вычисления дельт! Установите torchaudio. !!!")
    raise SystemExit("Остановка: Отсутствует torchaudio.")

# --- Визуализация и Интерактивность (IPython/Jupyter) ---
try:
    import ipywidgets as widgets
    from IPython.display import display, Audio, clear_output
    IPYWIDGETS_AVAILABLE = True
except ImportError:
    IPYWIDGETS_AVAILABLE = False
    print("Предупреждение: ipywidgets не найден. Интерактивные ячейки будут недоступны.")
    def display(*args, **kwargs): pass
    def Audio(*args, **kwargs): pass
    def clear_output(*args, **kwargs): pass

import matplotlib.pyplot as plt

# --- PyTorch ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR

# --- Метрики и Утилиты ---
try:
    import Levenshtein
    LEVENSHTEIN_AVAILABLE = True
except ImportError:
    LEVENSHTEIN_AVAILABLE = False
    print("!!! ПРЕДУПРЕЖДЕНИЕ: Библиотека Levenshtein не найдена. Метрика качества будет недоступна. Установите python-Levenshtein !!!")
    # Простая заглушка, если библиотека не найдена
    class Levenshtein:
        @staticmethod
        def distance(s1, s2): return max(len(s1), len(s2)) # Возвращает максимальную длину как штраф

from tqdm.notebook import tqdm # Используем версию для Jupyter
import itertools

# --- Логирование Экспериментов (MLflow) ---
try:
    import mlflow
    import mlflow.pytorch
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Предупреждение: mlflow не найден. Логирование экспериментов будет отключено.")
    # Заглушка для MLflow будет определена в Ячейке 12

# --- Настройка окружения и предупреждений ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("\n--- Статус ключевых библиотек ---")
print(f"PyTorch Версия: {torch.__version__}")
print(f"LibROSA Версия: {librosa.__version__}")
print(f"Torchaudio доступен: {TORCHAUDIO_AVAILABLE}")
print(f"NumPy Версия: {np.__version__}")
print(f"Pandas Версия: {pd.__version__}")
print(f"Levenshtein доступен: {LEVENSHTEIN_AVAILABLE}")
print(f"MLflow доступен: {MLFLOW_AVAILABLE}")
print(f"IPyWidgets доступен: {IPYWIDGETS_AVAILABLE}")
print("-" * 50)
# =============================================================================

--- Ячейка 1: Импорты и Базовая Настройка ---

--- Статус ключевых библиотек ---
PyTorch Версия: 2.5.1+cu121
LibROSA Версия: 0.11.0
Torchaudio доступен: True
NumPy Версия: 2.0.2
Pandas Версия: 2.2.3
Levenshtein доступен: True
MLflow доступен: True
IPyWidgets доступен: True
--------------------------------------------------


### Ячейка 2: Конфигурация и Параметры

In [185]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 2: Конфигурация и Параметры (ФИНАЛ: Grouped Conv g=2, Delta win=3)
# =============================================================================
print("--- Ячейка 2: Конфигурация и Параметры (ФИНАЛ: Grouped Conv g=2, Delta win=3) ---")

import json
from pathlib import Path
import random
import numpy as np
import torch

SEED = 42

# --- Пути ---
BASE_DIR = Path('.').resolve()
DATA_DIR = BASE_DIR
ZIP_PATH = DATA_DIR / 'morse_dataset.zip'
AUDIO_DIR_NAME = 'morse_dataset'
EXTRACTED_AUDIO_DIR = DATA_DIR / AUDIO_DIR_NAME / AUDIO_DIR_NAME
TRAIN_CSV_PATH = DATA_DIR / 'train.csv'
TEST_CSV_PATH = DATA_DIR / 'test.csv'
SAMPLE_SUB_PATH = DATA_DIR / 'sample_submission.csv'
OUTPUT_DIR = BASE_DIR / 'output_final_model' # Отдельная папка для финальной модели
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Базовая директория: {BASE_DIR}")
print(f"Ожидаемая директория аудио: {EXTRACTED_AUDIO_DIR}")
print(f"Директория для вывода: {OUTPUT_DIR}")

# --- Параметры обработки аудио (ФИНАЛ) ---
SR_FOR_CONFIG = 8000; HOP_LENGTH_CONST = 64; N_MELS_CONST = 12; N_FFT_CONST = 128; DELTA_WIN_CONST = 3
AUDIO_CONFIG = {
    "feature_type": "melspec_delta_win3_grouped2", # Тип признаков финальной модели
    "sample_rate": SR_FOR_CONFIG,
    "n_fft": N_FFT_CONST,
    "hop_length": HOP_LENGTH_CONST,
    "n_mels": N_MELS_CONST,
    "fmin": 0.0,
    "fmax": 4200.0,
    "power": 2.0,
    "apply_trimming": False, # Оставляем False
    "trim_top_db": 30,
    "delta_win_length": DELTA_WIN_CONST # Оптимальное окно для дельты
}
print(f"\nАудио параметры (ФИНАЛ):")
print(json.dumps(AUDIO_CONFIG, indent=2))

# --- Параметры Модели (ФИНАЛ: Grouped Conv g=2, CNN1=128, RNN2) ---
MODEL_CONFIG_FINAL = {
    "input_feature_dim": N_MELS_CONST * 2, # Вход = 2 * n_mels
    "cnn_out_channels": [128, 128, 128], # Первый слой 128 (64 для M, 64 для D1)
    "cnn_kernel_size": 9,
    "cnn_stride": 1,
    "cnn_padding": 'same',
    "cnn_pool_kernel": 2,
    "rnn_hidden_size": 512, # Полный размер
    "rnn_num_layers": 3,    # 2 слоя RNN
    "dropout_rate": 0.2,
    "activation_fn": "GELU",
    "classifier_type": "single",
    "num_feature_groups": 2 # Указываем 2 группы
}
# Проверка кратности первого канала
if MODEL_CONFIG_FINAL["cnn_out_channels"][0] % MODEL_CONFIG_FINAL["num_feature_groups"] != 0:
    raise ValueError(f"Первый cnn_out_channel ({MODEL_CONFIG_FINAL['cnn_out_channels'][0]}) должен быть кратен num_feature_groups ({MODEL_CONFIG_FINAL['num_feature_groups']})!")

print(f"\nПараметры модели (ФИНАЛ):")
print(json.dumps(MODEL_CONFIG_FINAL, indent=2))

# --- Параметры Обучения (ФИНАЛ) ---
TRAIN_CONFIG_FINAL = {
    "batch_size": 8, # Оставляем 8 для GTX 1050 Ti
    "num_workers": 0,
    "num_epochs": 80, # Увеличиваем для полного обучения (можно и больше, если время позволяет)
    "learning_rate": 1e-4, # Стандартный LR
    "div_factor": 10.0,
    "final_div_factor": 10000,
    "weight_decay": 1e-4,
    "optimizer": "AdamW",
    "early_stopping_patience": 12, # Увеличиваем терпимость
    "gradient_clip_norm": 2.0,
    "validation_split_ratio": 0.1,
    "base_seed": SEED,
    "batches_per_epoch": 0 # Полные эпохи
}
print(f"\nПараметры обучения (ФИНАЛ):")
print(json.dumps(TRAIN_CONFIG_FINAL, indent=2))

# --- Специальные Токены и Глобальные Переменные ---
PAD_TOKEN = '<pad>'; BLANK_TOKEN = '_'
PAD_IDX = -1; BLANK_IDX = -1; NUM_CLASSES_CTC = -1
char_to_index: Dict[str, int] = {}; index_to_char: Dict[int, str] = {}

# --- Шаблон Базового Суффикса Имени Файла (ФИНАЛ) ---
BASE_FILENAME_SUFFIX_FINAL = (
    f"SR{AUDIO_CONFIG['sample_rate'] // 1000}k_"
    f"Mel{N_MELS_CONST}x2win{DELTA_WIN_CONST}_fft{AUDIO_CONFIG['n_fft']}h{HOP_LENGTH_CONST}_f0-4k2_" # MelNx2win3
    f"CNN{MODEL_CONFIG_FINAL['cnn_out_channels'][0]}-{MODEL_CONFIG_FINAL['cnn_out_channels'][-1]}g{MODEL_CONFIG_FINAL['num_feature_groups']}_" # CNN128-128g2
    f"RNN{MODEL_CONFIG_FINAL['rnn_num_layers']}x{MODEL_CONFIG_FINAL['rnn_hidden_size']}_BiGRU_" # RNN2x512
    f"{MODEL_CONFIG_FINAL['activation_fn']}_Cls{MODEL_CONFIG_FINAL['classifier_type']}_"
    f"LR{TRAIN_CONFIG_FINAL['learning_rate']:.0e}_WD{TRAIN_CONFIG_FINAL['weight_decay']:.0e}"
)
print(f"\nШаблон базового суффикса (ФИНАЛ): {BASE_FILENAME_SUFFIX_FINAL}")

# --- Выбор устройства и Установка SEED ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nИспользуемое устройство: {device}")
if device.type == 'cuda':
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

def set_seed(seed_value: int):
    """Устанавливает seed для воспроизводимости."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    print(f"Установлен SEED = {seed_value}")

set_seed(TRAIN_CONFIG_FINAL['base_seed'])

print("\n--- Ячейка 2: Финальная конфигурация готова ---")
print("-" * 50)
# =============================================================================

--- Ячейка 2: Конфигурация и Параметры (ФИНАЛ: Grouped Conv g=2, Delta win=3) ---
Базовая директория: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder
Ожидаемая директория аудио: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset
Директория для вывода: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\output_final_model

Аудио параметры (ФИНАЛ):
{
  "feature_type": "melspec_delta_win3_grouped2",
  "sample_rate": 8000,
  "n_fft": 128,
  "hop_length": 64,
  "n_mels": 12,
  "fmin": 0.0,
  "fmax": 4200.0,
  "power": 2.0,
  "apply_trimming": false,
  "trim_top_db": 30,
  "delta_win_length": 3
}

Параметры модели (ФИНАЛ):
{
  "input_feature_dim": 24,
  "cnn_out_channels": [
    128,
    128,
    128
  ],
  "cnn_kernel_size": 9,
  "cnn_stride": 1,
  "cnn_padding": "same",
  "cnn_pool_kernel": 2,
  "rnn_hidden_size": 512,
  "rnn_num_layers": 3,
  "dropout_rate": 0.2,
  "activation_fn": "GELU",
  "classifier_type": "single

### Ячейка 3: Загрузка данных и распаковка

In [186]:
# =============================================================================
# Ячейка 3: Загрузка Метаданных и Распаковка Аудио
# =============================================================================
print("--- Ячейка 3: Загрузка метаданных и распаковка аудио ---")

# --- Проверка и Распаковка Архива ---
if not EXTRACTED_AUDIO_DIR.exists():
    print(f"Папка для аудио ({EXTRACTED_AUDIO_DIR}) не найдена.")
    if ZIP_PATH.is_file():
        print(f"Найден архив: {ZIP_PATH}. Распаковка...")
        try:
            with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
                zip_ref.extractall(DATA_DIR)
            print(f"Архив распакован в: {DATA_DIR}")
            # Проверяем, появилась ли ожидаемая папка после распаковки
            if not EXTRACTED_AUDIO_DIR.is_dir():
                 # Возможно, архив содержит папку с другим именем или без корневой папки
                 # Попробуем найти папку, начинающуюся с AUDIO_DIR_NAME
                 found_dirs = [d for d in DATA_DIR.iterdir() if d.is_dir() and d.name.startswith(AUDIO_DIR_NAME)]
                 if len(found_dirs) == 1:
                     EXTRACTED_AUDIO_DIR = found_dirs[0] # Обновляем путь
                     print(f"Обнаружена папка с аудио: {EXTRACTED_AUDIO_DIR}")
                 elif len(found_dirs) > 1:
                     print(f"!!! ПРЕДУПРЕЖДЕНИЕ: Найдено несколько папок, похожих на '{AUDIO_DIR_NAME}': {found_dirs}. Используется первая: {found_dirs[0]}")
                     EXTRACTED_AUDIO_DIR = found_dirs[0]
                 else:
                     raise FileNotFoundError(f"Не удалось найти папку '{AUDIO_DIR_NAME}' или похожую после распаковки в {DATA_DIR}.")
            else:
                 print(f"Папка с аудио найдена: {EXTRACTED_AUDIO_DIR}")
        except Exception as e:
            print(f"Критическая ошибка при распаковке: {e}")
            traceback.print_exc()
            raise SystemExit("Остановка.")
    else:
        print(f"Критическая ошибка: Архив {ZIP_PATH} не найден.")
        raise SystemExit("Остановка.")
else:
    print(f"Папка с аудио ({EXTRACTED_AUDIO_DIR}) уже существует. Распаковка пропускается.")

# --- Загрузка CSV файлов метаданных ---
print("\nЗагрузка CSV файлов...")
try:
    train_df = pd.read_csv(TRAIN_CSV_PATH)
    print(f"  Train DataFrame загружен: {len(train_df)} записей, Колонки: {train_df.columns.tolist()}")
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"  Test DataFrame загружен: {len(test_df)} записей, Колонки: {test_df.columns.tolist()}")
    try:
        sample_sub_df = pd.read_csv(SAMPLE_SUB_PATH)
        print(f"  Sample Submission загружен: {len(sample_sub_df)} записей.")
    except FileNotFoundError:
        print(f"  Предупреждение: Файл Sample Submission ({SAMPLE_SUB_PATH}) не найден.")
        sample_sub_df = None
except FileNotFoundError as e:
    print(f"Критическая ошибка: Не найден CSV файл: {e}.")
    raise SystemExit("Остановка.")
except Exception as e:
    print(f"Критическая ошибка при чтении CSV: {e}")
    traceback.print_exc()
    raise SystemExit("Остановка.")

# --- Проверка наличия файлов и колонок ---
print("\nПроверка данных...")
if 'id' not in train_df.columns or 'message' not in train_df.columns:
    raise ValueError("В train_df отсутствуют необходимые колонки 'id' или 'message'.")
if 'id' not in test_df.columns:
    raise ValueError("В test_df отсутствует необходимая колонка 'id'.")

# --- Выборочная проверка существования аудиофайлов ---
print("\nВыборочная проверка наличия аудиофайлов...")
if EXTRACTED_AUDIO_DIR.is_dir():
    num_check = 5
    if len(train_df) >= num_check:
        example_ids = train_df['id'].sample(num_check, random_state=SEED).tolist()
    else:
        example_ids = train_df['id'].tolist() # Проверяем все, если их мало

    missing_files = []
    for file_id in example_ids:
        expected_path = EXTRACTED_AUDIO_DIR / file_id
        if not expected_path.is_file():
            missing_files.append(file_id)

    if not missing_files:
        print(f"  Проверено {len(example_ids)} файлов - все найдены (например, {EXTRACTED_AUDIO_DIR / example_ids[0]}).")
    else:
        print(f"  !!! ПРЕДУПРЕЖДЕНИЕ: Не найдены файлы для ID: {missing_files} !!!")
        print(f"  Ожидаемый путь: {EXTRACTED_AUDIO_DIR}/<id>")
else:
    print(f"  Проверка невозможна: Папка {EXTRACTED_AUDIO_DIR} не существует или не является директорией.")

print("\n--- Ячейка 3: Загрузка данных завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 3: Загрузка метаданных и распаковка аудио ---
Папка с аудио (C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset) уже существует. Распаковка пропускается.

Загрузка CSV файлов...
  Train DataFrame загружен: 30000 записей, Колонки: ['id', 'message']
  Test DataFrame загружен: 5000 записей, Колонки: ['id']
  Sample Submission загружен: 5000 записей.

Проверка данных...

Выборочная проверка наличия аудиофайлов...
  Проверено 5 файлов - все найдены (например, C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset\2309.opus).

--- Ячейка 3: Загрузка данных завершена ---
--------------------------------------------------


### Ячейка 4: Создание словаря символов

In [187]:
# =============================================================================
# Ячейка 4: Создание Словаря Символов
# =============================================================================
print("--- Ячейка 4: Создание словаря символов ---")

# Проверка наличия train_df
if 'train_df' not in globals() or train_df is None:
    raise SystemExit("Остановка: train_df не найден. Выполните Ячейку 3.")
if 'message' not in train_df.columns:
    raise SystemExit("Остановка: В train_df отсутствует колонка 'message'.")

try:
    # Собираем все уникальные символы из обучающей выборки
    # Обрабатываем возможные NaN значения в 'message'
    all_texts = train_df['message'].fillna('').astype(str)
    unique_chars = sorted(list(set(char for text in all_texts for char in text)))

    # Создаем словари для преобразования
    char_to_index = {char: i for i, char in enumerate(unique_chars)}
    index_to_char = {i: char for char, i in char_to_index.items()}

    # Добавляем специальные токены BLANK (для CTC) и PAD (для выравнивания)
    # Убедимся, что они еще не существуют в словаре (маловероятно)
    if BLANK_TOKEN in char_to_index: print(f"Предупреждение: BLANK_TOKEN '{BLANK_TOKEN}' уже есть в данных!")
    if PAD_TOKEN in char_to_index: print(f"Предупреждение: PAD_TOKEN '{PAD_TOKEN}' уже есть в данных!")

    BLANK_IDX = len(char_to_index)
    PAD_IDX = len(char_to_index) + 1
    char_to_index[BLANK_TOKEN] = BLANK_IDX
    char_to_index[PAD_TOKEN] = PAD_IDX
    index_to_char[BLANK_IDX] = BLANK_TOKEN
    index_to_char[PAD_IDX] = PAD_TOKEN

    # Общее количество классов для выходного слоя модели (включая BLANK)
    NUM_CLASSES_CTC = BLANK_IDX + 1 # Количество классов = индекс BLANK + 1

    print(f"Найдено уникальных символов: {len(unique_chars)}")
    print(f"Символы: {''.join(unique_chars)}")
    print(f"Размер словаря (включая BLANK и PAD): {len(char_to_index)}")
    print(f"Индекс BLANK ('{BLANK_TOKEN}'): {BLANK_IDX}")
    print(f"Индекс PAD ('{PAD_TOKEN}'): {PAD_IDX}")
    print(f"Количество классов для CTC Loss: {NUM_CLASSES_CTC}")

except Exception as e:
    print(f"Критическая ошибка при создании словаря: {e}")
    traceback.print_exc()
    raise SystemExit("Остановка.")

# Финальная проверка
if not char_to_index or not index_to_char or BLANK_IDX == -1 or PAD_IDX == -1 or NUM_CLASSES_CTC <= 0:
    raise SystemExit("Остановка: Ошибка инициализации словаря или специальных токенов.")

print("\n--- Ячейка 4: Создание словаря завершено ---")
print("-" * 50)

--- Ячейка 4: Создание словаря символов ---
Найдено уникальных символов: 44
Символы:  #0123456789АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ
Размер словаря (включая BLANK и PAD): 46
Индекс BLANK ('_'): 44
Индекс PAD ('<pad>'): 45
Количество классов для CTC Loss: 45

--- Ячейка 4: Создание словаря завершено ---
--------------------------------------------------


### Ячейка 5: Класс MorseDataset

In [188]:
# =============================================================================
# Ячейка 5: Класс MorseDataset (ФИНАЛ: MelSpec + Delta win=3 + Grouped g=2)
# =============================================================================
print("--- Ячейка 5: Определение класса MorseDataset (ФИНАЛ: MelSpec + Delta win=3 + Grouped g=2) ---")

# Импорты и проверки зависимостей
import torch
import numpy as np
import pandas as pd
import librosa
import traceback
from torch.utils.data import Dataset
from pathlib import Path
from typing import Union, Tuple, List, Dict, Optional

# Проверка torchaudio (уже сделана в Ячейке 1)
if not TORCHAUDIO_AVAILABLE:
     raise SystemExit("Остановка: Отсутствует torchaudio, необходимый для этого датасета.")

# Проверка наличия глобальных переменных
if 'BLANK_IDX' not in globals() or 'PAD_IDX' not in globals(): raise ValueError("Индексы BLANK/PAD не инициализированы.")
if 'AUDIO_CONFIG' not in globals(): raise ValueError("AUDIO_CONFIG не определен.") # Используем финальный конфиг

class MorseDataset(Dataset):
    """
    Датасет для Морзе (Мел-Спектрограммы + Дельта win=3 для Grouped Conv g=2).
    Выполняет: Загрузка -> Ресэмплинг -> [Trimming] -> Мел-Спектрограмма -> Дельта (win=3) ->
               -> Независимая Z-Score Нормализация -> Стек (M+D1).
    """
    def __init__(self,
                 dataframe: pd.DataFrame,
                 audio_dir: Path,
                 char_to_index: Dict[str, int],
                 audio_config: Dict, # Принимает актуальный audio_config
                 model_input_feature_dim: int, # Ожидаемый размер = 2 * n_mels
                 is_train: bool = True):
        super().__init__()
        # Проверки типов аргументов
        if not isinstance(dataframe, pd.DataFrame): raise TypeError("dataframe должен быть pandas DataFrame")
        if not isinstance(audio_dir, Path): raise TypeError("audio_dir должен быть pathlib.Path")
        if not isinstance(char_to_index, dict): raise TypeError("char_to_index должен быть словарем")
        if not isinstance(audio_config, dict): raise TypeError("audio_config должен быть словарем")

        self.dataframe = dataframe.reset_index(drop=True)
        self.audio_dir = audio_dir
        self.char_to_index = char_to_index
        self.is_train = is_train
        self.audio_config = audio_config
        self.expected_feature_dim = model_input_feature_dim # Ожидаем 2 * n_mels

        # --- Извлечение параметров из audio_config ---
        try:
            self.feature_type = self.audio_config.get('feature_type', 'melspec_delta_win3_grouped2')
            if self.feature_type != 'melspec_delta_win3_grouped2':
                 raise ValueError(f"Этот датасет поддерживает только feature_type='melspec_delta_win3_grouped2'")

            self.sample_rate = int(self.audio_config['sample_rate'])
            self.n_fft = int(self.audio_config['n_fft'])
            self.hop_length = int(self.audio_config['hop_length'])
            self.n_mels = int(self.audio_config['n_mels'])
            self.power = float(self.audio_config.get('power', 2.0))
            self.fmin = float(self.audio_config.get('fmin', 0.0))
            self.fmax = float(self.audio_config.get('fmax', self.sample_rate / 2.0))
            self.apply_trimming = bool(self.audio_config.get('apply_trimming', False))
            self.trim_top_db = float(self.audio_config.get('trim_top_db', 30))
            self.delta_win_length = int(self.audio_config.get('delta_win_length', 3)) # Используем значение из конфига

            nyquist = self.sample_rate / 2.0
            if self.fmax > nyquist + 1:
                print(f"  Предупреждение Dataset: fmax ({self.fmax:.0f} Hz) > Найквиста ({nyquist:.0f} Hz).")

        except KeyError as e: raise ValueError(f"Отсутствует ключ в audio_config: {e}")
        except (TypeError, ValueError) as e: raise ValueError(f"Ошибка типа/значения в audio_config: {e}")

        # --- Валидация параметров ---
        if self.sample_rate <= 0: raise ValueError("sample_rate > 0")
        if self.n_fft <= 0: raise ValueError("n_fft > 0")
        if self.hop_length <= 0: raise ValueError("hop_length > 0")
        if self.n_mels <= 0: raise ValueError("n_mels > 0")
        if self.apply_trimming and self.trim_top_db <= 0: raise ValueError("trim_top_db > 0 при apply_trimming=True")
        if self.delta_win_length <= 0 or self.delta_win_length % 2 == 0:
             raise ValueError(f"delta_win_length ({self.delta_win_length}) должен быть положительным нечетным числом.")
        if self.n_mels * 2 != self.expected_feature_dim:
             raise ValueError(f"Размерность входа модели ({self.expected_feature_dim}) не равна 2 * n_mels ({self.n_mels})!")

        # --- Информационное сообщение (однократно) ---
        trim_status = f"Trimming ON (top_db={self.trim_top_db})" if self.apply_trimming else "Trimming OFF"
        print(f"MorseDataset (Final - GroupedInput g=2, Delta win={self.delta_win_length}): is_train={self.is_train}, SR={self.sample_rate}Hz, {trim_status}")
        print(f"  Признаки: MelSpec(n_fft={self.n_fft}, hop={self.hop_length}, n_mels={self.n_mels}) + Delta (win={self.delta_win_length})")
        print(f"  Нормализация: Z-Score (независимая для M, D1)")
        print(f"  Итоговый вход: ({2 * self.n_mels}, T)")

    def __len__(self) -> int:
        """Возвращает количество примеров в датасете."""
        return len(self.dataframe)

    def _normalize_feature(self, feature_matrix: np.ndarray) -> np.ndarray:
        """Стандартная Z-score нормализация для матрицы признаков (F, T)."""
        if not isinstance(feature_matrix, np.ndarray) or feature_matrix.size == 0:
            return np.array([[]], dtype=np.float32).reshape(feature_matrix.shape[0], 0) if feature_matrix.ndim == 2 else np.array([], dtype=np.float32)
        epsilon = 1e-8
        mean = np.mean(feature_matrix)
        std = np.std(feature_matrix)
        if std < epsilon:
            return np.zeros_like(feature_matrix, dtype=np.float32)
        return ((feature_matrix - mean) / (std + epsilon)).astype(np.float32)

    def _calculate_features(self, waveform_np: np.ndarray, file_id_for_log: str = "N/A") -> Optional[torch.Tensor]:
        """Вычисляет признаки: MelSpec -> Delta (win=3) -> Независимая Нормализация -> Стек (M+D1)."""
        empty_tensor = torch.empty((self.expected_feature_dim, 0), dtype=torch.float32)
        if not isinstance(waveform_np, np.ndarray) or waveform_np.size == 0:
            return empty_tensor
        try:
            processed_waveform = waveform_np.astype(np.float32)

            # 1. Вычисление Мел-спектрограммы
            mel_spectrogram = librosa.feature.melspectrogram(
                y=processed_waveform, sr=self.sample_rate, n_fft=self.n_fft, hop_length=self.hop_length,
                n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax, power=self.power
            ) # Shape: (n_mels, T)

            # Проверка на слишком короткий сигнал для вычисления дельт
            if mel_spectrogram.shape[1] < self.delta_win_length:
                 # print(f"Warning ({file_id_for_log}): Слишком короткий сигнал ({mel_spectrogram.shape[1]} < {self.delta_win_length}) для вычисления дельт. Пропуск.")
                 return empty_tensor

            # 2. Конвертация в децибелы
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max, top_db=None)

            # 3. Вычисление Дельты (используем torchaudio с заданным win_length)
            mel_spec_tensor = torch.from_numpy(mel_spectrogram_db) # (n_mels, T)
            # compute_deltas ожидает (..., time, freq), поэтому пермутируем
            mel_spec_permuted = mel_spec_tensor.permute(1, 0) # (T, n_mels)
            deltas = torchaudio.functional.compute_deltas(mel_spec_permuted, win_length=self.delta_win_length) # (T, n_mels)
            # Возвращаем к (n_mels, T) и конвертируем в NumPy для нормализации
            deltas = deltas.permute(1, 0).numpy() # (n_mels, T)

            # 4. Независимая Нормализация Z-Score
            norm_melspec = self._normalize_feature(mel_spectrogram_db)
            norm_deltas = self._normalize_feature(deltas)

            # Проверка размерностей после нормализации
            if not (norm_melspec.shape == norm_deltas.shape):
                 print(f"ERROR ({file_id_for_log}): Несовпадение форм после нормализации! M:{norm_melspec.shape}, D1:{norm_deltas.shape}")
                 return None
            if norm_melspec.shape[1] == 0: # Проверка на пустые признаки
                 # print(f"Debug ({file_id_for_log}): Пустой тензор признаков после обработки.")
                 return empty_tensor

            # 5. Сборка итогового тензора (M+D1)
            features_np = np.vstack([norm_melspec, norm_deltas]).astype(np.float32) # (2 * n_mels, T)
            features_tensor = torch.from_numpy(features_np)

            # Финальная проверка на NaN/Inf
            if not torch.isfinite(features_tensor).all():
                print(f"WARNING ({file_id_for_log}): NaN/Inf в финальном тензоре признаков! Замена на 0.")
                features_tensor = torch.nan_to_num(features_tensor, nan=0.0, posinf=0.0, neginf=0.0)

            # Проверка итоговой размерности
            if features_tensor.shape[0] != self.expected_feature_dim:
                 print(f"ERROR ({file_id_for_log}): Неожиданная итоговая размерность признаков! Ожидалось {self.expected_feature_dim}, получено {features_tensor.shape[0]}")
                 return None

            return features_tensor # (2 * n_mels, T)

        except Exception as e:
            print(f"CRITICAL ERROR в _calculate_features (Final - GroupedInput g=2, Delta win={self.delta_win_length}) ({file_id_for_log}): {e}")
            traceback.print_exc(limit=1)
            return None

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, str], Tuple[None, Optional[str]]]:
        """Загрузка, обработка (Мел-Спектрограмма + Дельта win=3) и возврат примера."""
        if not (0 <= index < len(self.dataframe)):
            return None, (f"InvalidIndex_{index}" if not self.is_train else None)
        try:
            row = self.dataframe.iloc[index]
            file_id = row['id']
            audio_path = self.audio_dir / file_id
        except Exception as e:
            print(f"Error get item data index {index}: {e}")
            return None, (f"DataAccessError_{index}" if not self.is_train else None)

        # 1. Загрузка и Ресэмплинг
        waveform_np: Optional[np.ndarray] = None
        try:
            waveform_np, _ = librosa.load(audio_path, sr=self.sample_rate, mono=True)
            if waveform_np is None or waveform_np.size == 0:
                # print(f"Warning ({file_id}): Файл пуст или не удалось загрузить.")
                return None, (file_id if not self.is_train else None)
        except FileNotFoundError:
            print(f"ERROR ({file_id}): Файл не найден {audio_path}")
            return None, (file_id if not self.is_train else None)
        except Exception as e:
            print(f"ERROR ({file_id}): Ошибка загрузки librosa: {e}")
            traceback.print_exc(limit=1)
            return None, (file_id if not self.is_train else None)

        # 2. Обрезка тишины (Trimming) - если включено
        if self.apply_trimming:
            try:
                waveform_trimmed, _ = librosa.effects.trim(waveform_np, top_db=self.trim_top_db, frame_length=512, hop_length=128)
                if waveform_trimmed is not None and waveform_trimmed.size > 0:
                    waveform_np = waveform_trimmed
                # else: Оставляем исходный, если trim удалил всё
            except Exception as e_trim:
                print(f"ERROR ({file_id}): Ошибка librosa.effects.trim: {e_trim}. Используется исходный сигнал.")

        # 3. Вычисление признаков (Мел-Спектрограмма + Дельта -> Нормализация -> Стек)
        features: Optional[torch.Tensor] = self._calculate_features(waveform_np, file_id)

        # Проверка на None или пустой тензор (0 временных шагов)
        if features is None or features.shape[1] == 0:
            # print(f"Warning ({file_id}): Пустой тензор признаков после обработки.")
            return None, (file_id if not self.is_train else None)

        # 4. Подготовка цели (для train) или возврат ID (для test)
        if self.is_train:
            message_text = str(row.get('message', ''))
            target_indices = [self.char_to_index.get(c) for c in message_text if c in self.char_to_index]
            if not target_indices:
                # print(f"Warning ({file_id}): Пустой или невалидный таргет: '{message_text}'")
                return None, None # Возвращаем None, None чтобы collate_fn его отфильтровал
            target_tensor = torch.tensor(target_indices, dtype=torch.long)
            return features, target_tensor # (F, T), (Target_Len)
        else:
            # Для тестового режима возвращаем признаки и ID файла
            return features, file_id # (F, T), str

print("\n--- Ячейка 5: Определение MorseDataset (ФИНАЛ) завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 5: Определение класса MorseDataset (ФИНАЛ: MelSpec + Delta win=3 + Grouped g=2) ---

--- Ячейка 5: Определение MorseDataset (ФИНАЛ) завершено ---
--------------------------------------------------


### Ячейка 6: Функция collate_fn

In [189]:
# =============================================================================
# Ячейка 6: Функция collate_fn (Сборка Батчей)
# =============================================================================
print("--- Ячейка 6: Определение функции collate_fn ---")

# Импорты и проверки зависимостей
import torch
from torch.nn.utils.rnn import pad_sequence
from typing import List, Tuple, Optional, Union

if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не инициализирован!")

def collate_fn(batch: List[Tuple[Optional[torch.Tensor], Optional[Union[torch.Tensor, str]]]]) \
    -> Optional[Tuple[torch.Tensor, Union[torch.Tensor, List[str]], torch.Tensor, Optional[torch.Tensor]]]:
    """
    Собирает батч данных из списка кортежей, возвращаемых MorseDataset.
    Фильтрует некорректные примеры (где признаки или таргет/ID равны None).
    Выполняет паддинг признаков и таргетов (для train/val).
    Возвращает батч признаков, батч таргетов (или ID), длины признаков и длины таргетов.
    """
    # 1. Фильтрация некорректных/пустых примеров
    valid_batch = [item for item in batch if
                   item[0] is not None and item[1] is not None and item[0].shape[1] > 0]

    if not valid_batch:
        # print("Warning: collate_fn получил пустой валидный батч.")
        return None # Если после фильтрации батч пуст

    # Определяем, содержит ли батч таргеты (тензоры) или ID (строки)
    is_train_or_val_batch = isinstance(valid_batch[0][1], torch.Tensor)

    # 2. Разделение признаков и таргетов/ID
    # Признаки имеют форму (F, T), пермутируем в (T, F) для pad_sequence
    features_list = [item[0].permute(1, 0) for item in valid_batch] # Список тензоров (Ti, F)
    targets_or_ids_list = [item[1] for item in valid_batch] # Список тензоров таргетов или строк ID

    # 3. Паддинг признаков
    # batch_first=True -> (B, T_max, F)
    features_padded_time_first = pad_sequence(features_list, batch_first=True, padding_value=0.0)
    # Возвращаем к формату (B, F, T_max), ожидаемому моделью (Conv1d)
    features_padded = features_padded_time_first.permute(0, 2, 1)

    # 4. Расчет длин признаков (до паддинга)
    # Длина по временной оси (исходная T)
    feature_lengths = torch.tensor([f.shape[0] for f in features_list], dtype=torch.long)

    # 5. Обработка таргетов (для train/val) или ID (для test)
    if is_train_or_val_batch:
        # Батч содержит таргеты (тензоры)
        targets_list: List[torch.Tensor] = targets_or_ids_list
        # Паддинг таргетов
        targets_padded: torch.Tensor = pad_sequence(targets_list, batch_first=True, padding_value=PAD_IDX)
        # Расчет длин таргетов (до паддинга)
        target_lengths: torch.Tensor = torch.tensor([len(t) for t in targets_list], dtype=torch.long)
        # Возвращаем данные для обучения/валидации
        return features_padded, targets_padded, feature_lengths, target_lengths
    else:
        # Батч содержит ID файлов (строки)
        file_ids: List[str] = targets_or_ids_list
        # Возвращаем данные для инференса (таргетов и их длин нет)
        return features_padded, file_ids, feature_lengths, None

print("Функция collate_fn определена.")
print("\n--- Ячейка 6: Определение collate_fn завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 6: Определение функции collate_fn ---
Функция collate_fn определена.

--- Ячейка 6: Определение collate_fn завершено ---
--------------------------------------------------


### Ячейка 7: Модель MorseRecognizer

In [190]:
# =============================================================================
# Ячейка 7: Модель MorseRecognizer (ФИНАЛ: Grouped Conv g=2 для M+D1)
# =============================================================================
print("--- Ячейка 7: Определение модели MorseRecognizer (ФИНАЛ: Grouped Conv g=2 для M+D1) ---")

# Импорты и проверки зависимостей
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Union, Tuple, List, Dict, Optional

# Проверка наличия глобальных переменных
if 'MODEL_CONFIG_FINAL' not in globals(): raise ValueError("MODEL_CONFIG_FINAL не определен!")
if 'NUM_CLASSES_CTC' not in globals() or NUM_CLASSES_CTC <= 0: raise ValueError("NUM_CLASSES_CTC не корректен!")

class MorseRecognizer(nn.Module):
    """
    Модель для распознавания Морзе (CNN + BiGRU). ФИНАЛЬНАЯ ВЕРСИЯ.
    Использует Grouped Convolution (groups=2) в первом слое для раздельной обработки
    Mel-спектрограммы и ее первой дельты (win=3), поданных как стек каналов.
    """
    def __init__(self, num_classes_ctc: int, input_feature_dim: int, # Ожидается 2 * n_mels
                 cnn_out_channels: List[int], cnn_kernel_size: int, cnn_stride: int,
                 cnn_padding: Union[int, str], cnn_pool_kernel: int,
                 rnn_hidden_size: int, rnn_num_layers: int, dropout_rate: float,
                 activation_fn: str = "GELU", classifier_type: str = "single",
                 num_feature_groups: int = 2): # <-- Устанавливаем groups=2 по умолчанию
        super().__init__()
        # Проверка, что размерность входа кратна количеству групп
        if input_feature_dim % num_feature_groups != 0:
             raise ValueError(f"input_feature_dim ({input_feature_dim}) должен быть кратен num_feature_groups ({num_feature_groups})")
        # Проверка, что num_feature_groups соответствует ожиданиям (2)
        if num_feature_groups != 2:
             print(f"Warning: num_feature_groups={num_feature_groups}, но ожидалось 2 для M+D1.")

        self.input_feature_dim = input_feature_dim
        self.num_feature_groups = num_feature_groups # Должно быть 2
        self._time_reduction_factor = 1.0
        cnn_layers = []
        in_channels = input_feature_dim

        try:
            ActivationLayer = getattr(nn, activation_fn)
        except AttributeError:
            print(f"Warning: Activation '{activation_fn}' не найдена. Используется GELU.")
            ActivationLayer = nn.GELU

        # CNN Extractor
        for i, out_channels in enumerate(cnn_out_channels):
            # Устанавливаем группы только для первого слоя
            current_groups = self.num_feature_groups if i == 0 else 1
            # Убедимся, что out_channels кратно группам (для первого слоя)
            if i == 0 and out_channels % self.num_feature_groups != 0:
                 raise ValueError(f"cnn_out_channels[0] ({out_channels}) должен быть кратен num_feature_groups ({self.num_feature_groups}) для Grouped Conv")

            layer = nn.Sequential(
                nn.Conv1d(
                    in_channels, out_channels, cnn_kernel_size, cnn_stride,
                    padding=cnn_padding,
                    groups=current_groups # <-- Устанавливаем группы (2 для первого слоя)
                ),
                nn.BatchNorm1d(out_channels),
                ActivationLayer(),
                nn.MaxPool1d(cnn_pool_kernel),
                nn.Dropout(dropout_rate)
            )
            cnn_layers.append(layer)
            in_channels = out_channels # Для следующего слоя in_channels = out_channels предыдущего
            self._time_reduction_factor *= cnn_pool_kernel
        self.cnn_extractor = nn.Sequential(*cnn_layers)
        self.cnn_output_dim = in_channels

        # RNN (BiGRU)
        self.rnn = nn.GRU(
            input_size=self.cnn_output_dim, # Вход RNN = выход CNN
            hidden_size=rnn_hidden_size,
            num_layers=rnn_num_layers, # Используем значение из конфига (2)
            batch_first=True, # Ожидает (B, T, F_cnn)
            bidirectional=True, # Используем BiGRU
            dropout=dropout_rate if rnn_num_layers > 1 else 0.0
        )
        rnn_output_dim = rnn_hidden_size * 2 # Учитываем двунаправленность

        # Classifier
        self.classifier_type = classifier_type
        if self.classifier_type == "double":
            intermediate_dim = rnn_output_dim
            self.classifier = nn.Sequential(
                nn.Linear(rnn_output_dim, intermediate_dim),
                ActivationLayer(),
                nn.Dropout(dropout_rate),
                nn.Linear(intermediate_dim, num_classes_ctc)
            )
            classifier_str = f"DoubleLinear({rnn_output_dim}->{intermediate_dim}->{num_classes_ctc})"
        elif self.classifier_type == "single":
            self.classifier = nn.Linear(rnn_output_dim, num_classes_ctc)
            classifier_str = f"SingleLinear({rnn_output_dim}->{num_classes_ctc})"
        else:
            raise ValueError(f"Неизвестный classifier_type: {self.classifier_type}. Допустимы 'single', 'double'.")

        # Вывод архитектуры при инициализации (можно раскомментировать для отладки)
        # print(f"Архитектура MorseRecognizer (Final - Grouped Conv g={self.num_feature_groups}):")
        # print(f"  Input Feature Dim: {self.input_feature_dim} ({self.input_feature_dim // self.num_feature_groups} per group)")
        # print(f"  CNN (1D): {len(cnn_out_channels)} layers, OutChannels={cnn_out_channels}, Kernel={cnn_kernel_size}, Pool={cnn_pool_kernel}")
        # print(f"       Grouped Conv (Layer 0): groups={self.num_feature_groups}")
        # print(f"       CNN Output Dim={self.cnn_output_dim}, Time Reduction Factor={self._time_reduction_factor:.1f}x")
        # print(f"  RNN: BiGRU, Layers={rnn_num_layers}, Hidden Size={rnn_hidden_size}")
        # print(f"       RNN Output Dim={rnn_output_dim}")
        # print(f"  Activation: {activation_fn}")
        # print(f"  Classifier: {classifier_str}")
        # print(f"  Dropout: {dropout_rate}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Input: (B, F, T_in) - где F = 2 * n_mels
        -> CNN (Layer 0: Grouped Conv g=2) -> (B, C_cnn, T_red)
        -> Permute -> (B, T_red, C_cnn)
        -> RNN -> (B, T_red, H*2)
        -> Classifier -> (B, T_red, N_classes)
        -> Permute -> Output: (T_red, B, N_classes) - для CTC Loss
        """
        # Проверка размерности входа
        if x.shape[1] != self.input_feature_dim:
            raise ValueError(f"Input feature dim mismatch! Expected {self.input_feature_dim}, got {x.shape[1]}. Shape: {x.shape}")

        x = self.cnn_extractor(x)        # (B, C_cnn, T_reduced)
        x = x.permute(0, 2, 1)           # (B, T_reduced, C_cnn) - готовим для RNN (batch_first=True)
        x_rnn, _ = self.rnn(x)           # (B, T_reduced, Hidden*2)
        logits = self.classifier(x_rnn)  # (B, T_reduced, NumClasses)
        # Готовим для CTC Loss: (Time, Batch, Classes)
        logits = logits.permute(1, 0, 2) # (T_reduced, B, NumClasses)
        return logits

    def get_time_reduction_factor(self) -> float:
        """Возвращает фактор уменьшения временной размерности после CNN."""
        return self._time_reduction_factor

# --- Создание и Проверка Экземпляра Модели ---
model_created_successfully = False
model_check = None; dummy_input = None; dummy_output = None
try:
    print("\nСоздание экземпляра модели для проверки (ФИНАЛ)...")
    # Используем финальный конфиг из Ячейки 2
    temp_model_config_check = MODEL_CONFIG_FINAL.copy()

    model_check = MorseRecognizer(
        num_classes_ctc=NUM_CLASSES_CTC,
        **temp_model_config_check # Передаем финальный конфиг
    ).to(device)
    total_params = sum(p.numel() for p in model_check.parameters() if p.requires_grad)
    print(f"Модель '{type(model_check).__name__}' создана ({total_params:,} параметров) на {device}.")

    print("\nПроверка forward pass..."); model_check.eval()
    dummy_batch_size = 2; dummy_time_steps = 500
    # Используем правильную размерность входа
    dummy_input = torch.randn(dummy_batch_size, temp_model_config_check['input_feature_dim'], dummy_time_steps).to(device)
    with torch.no_grad(): dummy_output = model_check(dummy_input)
    print(f"  Вход: {dummy_input.shape}, Выход: {dummy_output.shape}")
    expected_time_dim = int(dummy_time_steps / model_check.get_time_reduction_factor())
    if abs(dummy_output.shape[0] - expected_time_dim) > 2: print(f"  ПРЕДУПРЕЖДЕНИЕ: Неожиданная длина выхода! Ожидалось ~{expected_time_dim}, получено {dummy_output.shape[0]}.")
    assert dummy_output.shape[1] == dummy_batch_size, "Batch size mismatch!"
    assert dummy_output.shape[2] == NUM_CLASSES_CTC, "Num classes mismatch!"
    print("  Размерности выхода (T_red, B, C) корректны."); model_created_successfully = True
except Exception as e: print(f"\n!!! КРИТИЧЕСКАЯ ОШИБКА при создании/проверке модели: {e} !!!"); traceback.print_exc()
finally: # Очистка
    if 'model_check' in locals(): del model_check
    if 'dummy_input' in locals(): del dummy_input
    if 'dummy_output' in locals(): del dummy_output
    if torch.cuda.is_available(): torch.cuda.empty_cache()

if not model_created_successfully: raise SystemExit("Остановка: Не удалось создать/проверить модель.")

print("\n--- Ячейка 7: Определение и проверка модели (ФИНАЛ) завершены ---")
print("-" * 50)
# =============================================================================

--- Ячейка 7: Определение модели MorseRecognizer (ФИНАЛ: Grouped Conv g=2 для M+D1) ---

Создание экземпляра модели для проверки (ФИНАЛ)...
Модель 'MorseRecognizer' создана (11,777,709 параметров) на cuda.

Проверка forward pass...
  Вход: torch.Size([2, 24, 500]), Выход: torch.Size([62, 2, 45])
  Размерности выхода (T_red, B, C) корректны.

--- Ячейка 7: Определение и проверка модели (ФИНАЛ) завершены ---
--------------------------------------------------


### Ячейка 8: Loss, Optimizer, Scheduler

In [191]:
# =============================================================================
# Ячейка 8: Настройка Loss, Optimizer, Scheduler
# =============================================================================
print("--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---")

# Проверка зависимостей
if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не инициализирован!")
if 'TRAIN_CONFIG_FINAL' not in globals(): raise ValueError("TRAIN_CONFIG_FINAL не определен! Убедитесь, что Ячейка 2 выполнена.")
if 'optim' not in globals(): import torch.optim as optim
if 'nn' not in globals(): import torch.nn as nn
if 'OneCycleLR' not in globals(): from torch.optim.lr_scheduler import OneCycleLR

# --- Функция Потерь ---
# Используем CTCLoss, стандарт для задач sequence-to-sequence без выравнивания
criterion = nn.CTCLoss(blank=BLANK_IDX, reduction='mean', zero_infinity=True)
print(f"Функция потерь: CTCLoss (blank={BLANK_IDX}, reduction='mean', zero_infinity=True)")

# --- Оптимизатор и Планировщик (параметры из TRAIN_CONFIG_FINAL) ---
# Сами объекты optimizer и scheduler будут создаваться внутри run_training_pipeline
optimizer_name = TRAIN_CONFIG_FINAL.get('optimizer', 'AdamW').lower()
lr = TRAIN_CONFIG_FINAL['learning_rate']
wd = TRAIN_CONFIG_FINAL['weight_decay']
print(f"\nОптимизатор: {optimizer_name.upper()} (LR={lr:.1e}, WD={wd:.1e}) - будет создан в пайплайне")

div_f = TRAIN_CONFIG_FINAL.get('div_factor', 25.0)
final_div_f = TRAIN_CONFIG_FINAL.get('final_div_factor', 1e4)
print(f"Планировщик: OneCycleLR (div={div_f}, final_div={final_div_f}) - будет создан в пайплайне")

print("\n--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---
Функция потерь: CTCLoss (blank=44, reduction='mean', zero_infinity=True)

Оптимизатор: ADAMW (LR=1.0e-04, WD=1.0e-04) - будет создан в пайплайне
Планировщик: OneCycleLR (div=10.0, final_div=10000) - будет создан в пайплайне

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---
--------------------------------------------------


### Ячейка 9: Функции Декодирования и Метрики

In [192]:
# =============================================================================
# Ячейка 9: Функции Декодирования (Greedy) и Метрики (Levenshtein)
# =============================================================================
print("--- Ячейка 9: Определение функций декодирования и метрики ---")

# Импорты и проверки зависимостей
import torch
import numpy as np
from typing import List, Dict, Tuple

if 'index_to_char' not in globals(): raise ValueError("index_to_char не определен!")
if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не определен!")
if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не определен!")
if not LEVENSHTEIN_AVAILABLE: print("Предупреждение: Levenshtein не найден, используется заглушка.")
else: import Levenshtein # Импортируем, если доступен

# --- Greedy CTC Decoding ---
def ctc_greedy_decode(logits: torch.Tensor, index_to_char_map: Dict[int, str], blank_idx: int) -> List[str]:
    """
    Жадное CTC декодирование батча логитов.
    Input: logits (Time, Batch, Classes) - выход модели после log_softmax не нужен.
    Output: List[str] - список декодированных строк для батча.
    """
    decoded_batch = []
    # Находим наиболее вероятный индекс для каждого временного шага в каждом примере батча
    # Используем .detach() для экономии памяти, так как градиенты здесь не нужны
    best_path = torch.argmax(logits.detach(), dim=2) # (Time, Batch)
    best_path_np = best_path.cpu().numpy()

    # Итерируем по каждому примеру в батче
    for i in range(best_path_np.shape[1]):
        sequence_indices = best_path_np[:, i]
        # 1. Схлопываем повторяющиеся символы
        collapsed_indices = [idx for j, idx in enumerate(sequence_indices) if j == 0 or idx != sequence_indices[j-1]]
        # 2. Удаляем BLANK символы
        final_indices = [idx for idx in collapsed_indices if idx != blank_idx]
        # 3. Преобразуем индексы в символы
        decoded_string = "".join([index_to_char_map.get(idx, '?') for idx in final_indices]) # '?' для неизвестных индексов
        decoded_batch.append(decoded_string)
    return decoded_batch

# --- Levenshtein Distance ---
def calculate_levenshtein(predictions: List[str], targets_padded: torch.Tensor, target_lengths: torch.Tensor,
                          index_to_char_map: Dict[int, str], pad_idx: int) -> Tuple[float, List[Tuple[str, str]]]:
    """
    Вычисляет среднее расстояние Левенштейна для батча и возвращает пары (предсказание, реальность).
    Input:
        predictions: List[str] - список предсказанных строк.
        targets_padded: torch.Tensor (Batch, MaxTargetLen) - тензор реальных таргетов с паддингом.
        target_lengths: torch.Tensor (Batch) - тензор реальных длин таргетов.
        index_to_char_map: Dict[int, str] - словарь для преобразования индексов в символы.
        pad_idx: int - индекс PAD токена.
    Output:
        Tuple[float, List[Tuple[str, str]]] - среднее расстояние Левенштейна, список пар (предсказание, реальность).
    """
    total_distance = 0.0
    num_valid_pairs = 0
    decoded_pairs = [] # Список для хранения пар (предсказание, реальность)
    targets_np = targets_padded.cpu().numpy()
    target_lengths_np = target_lengths.cpu().numpy()
    batch_size = targets_padded.shape[0]

    # Проверка соответствия размеров предсказаний и таргетов
    if len(predictions) != batch_size:
        print(f"Warning: Levenshtein size mismatch! Preds:{len(predictions)}, Targets:{batch_size}")
        return float('inf'), [] # Возвращаем бесконечность и пустой список

    # Итерируем по каждому примеру в батче
    for i in range(batch_size):
        real_target_len = target_lengths_np[i]
        pred_str = predictions[i]

        # Получаем реальную строку таргета, удаляя паддинг
        if real_target_len <= 0:
            target_str = ""
        else:
            target_indices = targets_np[i, :real_target_len]
            target_str = "".join([index_to_char_map.get(idx, '?') for idx in target_indices if idx != pad_idx])

        try:
            # Вычисляем расстояние Левенштейна
            dist = Levenshtein.distance(pred_str, target_str)
        except Exception as e:
            # Обработка возможных ошибок в Levenshtein
            print(f"Levenshtein Error: ('{pred_str}', '{target_str}'). {e}")
            dist = max(len(pred_str), len(target_str)) # Используем максимальную длину как штраф

        total_distance += dist
        num_valid_pairs += 1
        decoded_pairs.append((pred_str, target_str)) # Добавляем пару

    # Вычисляем среднее расстояние
    mean_levenshtein = total_distance / num_valid_pairs if num_valid_pairs > 0 else float('inf')
    return mean_levenshtein, decoded_pairs

print("Функции ctc_greedy_decode и calculate_levenshtein определены.")
print("\n--- Ячейка 9: Определение функций декодирования и метрики завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 9: Определение функций декодирования и метрики ---
Функции ctc_greedy_decode и calculate_levenshtein определены.

--- Ячейка 9: Определение функций декодирования и метрики завершено ---
--------------------------------------------------


### Ячейка 10: Функции Обучения и Валидации Эпохи

In [193]:
# =============================================================================
# Ячейка 10: Функции Обучения и Валидации Эпохи
# =============================================================================
print("--- Ячейка 10: Определение функций обучения и валидации эпохи ---")

# Импорты и проверки зависимостей
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import traceback
import itertools
from tqdm.notebook import tqdm
from typing import Optional, Tuple, List, Dict

# Проверка наличия функций из Ячейки 9
if 'ctc_greedy_decode' not in globals() or 'calculate_levenshtein' not in globals():
    raise NameError("Функции ctc_greedy_decode или calculate_levenshtein не определены!")

# --- Функция Обучения Одной Эпохи ---
def train_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, optimizer: optim.Optimizer,
                scheduler: Optional[optim.lr_scheduler._LRScheduler], device: torch.device, epoch_num: int, total_epochs: int,
                index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int, grad_clip_norm: float,
                batches_per_epoch: int = 0) -> Tuple[float, float, float]:
    """ Выполняет одну эпоху обучения модели. """
    model.train() # Переключаем модель в режим обучения
    running_loss = 0.0
    total_lev_dist = 0.0
    total_lr = 0.0
    num_batches_processed = 0
    total_samples = 0

    # Получаем фактор сжатия времени из модели (для расчета длин для CTC)
    try:
        time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError:
        print("Warning: Метод get_time_reduction_factor() не найден в модели. Используется time_factor=1.0")
        time_factor = 1.0

    # Определяем итератор и количество батчей для обработки
    total_batches_in_loader = len(dataloader)
    iterator = dataloader
    num_batches_to_process = total_batches_in_loader
    if batches_per_epoch > 0 and batches_per_epoch < total_batches_in_loader:
         iterator = itertools.islice(dataloader, batches_per_epoch)
         num_batches_to_process = batches_per_epoch
    if num_batches_to_process == 0:
        print("Warning: Нет батчей для обработки в train_epoch.")
        return 0.0, float('inf'), 0.0

    # Создаем прогресс-бар
    pbar = tqdm(iterator, total=num_batches_to_process, desc=f"Эпоха {epoch_num}/{total_epochs} [Тренировка]", leave=False, ncols=1000)

    # Итерация по батчам
    for batch_idx, batch_data in enumerate(pbar):
        if batch_data is None: continue # Пропускаем батч, если collate_fn вернул None
        features, targets, feature_lengths, target_lengths = batch_data
        if features is None or targets is None or feature_lengths is None or target_lengths is None: continue

        batch_size = features.size(0)
        if batch_size == 0: continue

        # Перемещаем данные на устройство
        features = features.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        feature_lengths_cpu = feature_lengths.cpu() # Оставляем на CPU для CTC Loss
        target_lengths_cpu = target_lengths.cpu()   # Оставляем на CPU для CTC Loss

        # Рассчитываем длины выхода модели для CTC Loss
        input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)

        loss_value = float('inf')
        lev_dist_batch = float('inf')
        current_lr = optimizer.param_groups[0]['lr']

        try:
            optimizer.zero_grad() # Обнуляем градиенты

            # Прямой проход через модель
            logits = model(features) # Ожидаемый выход: (T_red, B, C)

            if logits.shape[1] != batch_size:
                print(f"\n!!! ERROR (Train): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip batch.")
                continue

            # Применяем log_softmax для CTC Loss
            log_probs = F.log_softmax(logits, dim=2) # (T_red, B, C)

            # Убеждаемся, что длины входа для CTC не превышают реальную длину выхода модели
            output_length = log_probs.shape[0]
            input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)

            # Убеждаемся, что длины таргетов не превышают размерность тензора таргетов
            target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])

            # Вычисляем CTC Loss, обрабатывая случай нулевых длин таргетов
            valid_target_mask = target_lengths_clamped > 0
            if not torch.all(valid_target_mask):
                 log_probs_valid = log_probs[:, valid_target_mask, :]
                 targets_valid = targets[valid_target_mask, :]
                 input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]
                 target_lengths_valid = target_lengths_clamped[valid_target_mask]
                 # Считаем loss только если есть валидные примеры
                 loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
            else:
                 loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)

            # Проверка на NaN/Inf в loss
            if not torch.isfinite(loss):
                print(f"\n!!! WARNING (Train): NaN/Inf loss на батче {batch_idx}! Пропуск шага оптимизатора.")
                optimizer.zero_grad()
                loss_value = 30.0 # Штраф для логгирования
                lev_dist_batch = 30.0
                continue # Переходим к следующему батчу

            loss_value = loss.item()

            # Обратный проход и шаг оптимизатора
            if loss.requires_grad:
                loss.backward()
                if grad_clip_norm > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip_norm)
                optimizer.step()
                if scheduler:
                    scheduler.step() # OneCycleLR обновляется после каждого батча

            # Вычисление метрики Levenshtein (без градиентов)
            # Делаем это после шага оптимизатора, чтобы не замедлять цикл
            decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx) # logits уже посчитаны
            lev_dist_batch, _ = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
            if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0 # Ограничиваем для логгирования

        except RuntimeError as e:
             if "CUDA out of memory" in str(e):
                 print("\n!!! CUDA Out of Memory (Train) !!! Попробуйте уменьшить batch_size.")
                 raise e # Перевыбрасываем ошибку
             else:
                 print(f"\n!!! RuntimeError (Train) на батче {batch_idx}: {e} !!!")
                 traceback.print_exc(limit=1)
                 loss_value = 30.0; lev_dist_batch = 30.0
        except Exception as e:
             print(f"\n!!! Error (Train) на батче {batch_idx}: {e} !!!")
             traceback.print_exc(limit=1)
             loss_value = 30.0; lev_dist_batch = 30.0

        # Обновление статистики эпохи
        running_loss += loss_value * batch_size
        total_lev_dist += lev_dist_batch * batch_size
        total_samples += batch_size
        total_lr += current_lr
        num_batches_processed += 1

        # Обновление прогресс-бара
        pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}', lr=f'{current_lr:.2e}')

    pbar.close() # Закрываем прогресс-бар

    # Вычисляем средние значения за эпоху
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')
    avg_lr = total_lr / num_batches_processed if num_batches_processed > 0 else 0.0

    return avg_loss, avg_lev, avg_lr

# --- Функция Валидации Одной Эпохи ---
def validate_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, device: torch.device,
                   index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int
                  ) -> Tuple[float, float, List[Tuple[str, str]]]:
    """ Выполняет одну эпоху валидации модели. """
    model.eval() # Переключаем модель в режим оценки
    running_loss = 0.0
    total_lev_dist = 0.0
    total_samples = 0
    all_decoded_pairs = [] # Список для примеров декодирования

    # Получаем фактор сжатия времени
    try: time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError: time_factor = 1.0

    # Создаем прогресс-бар
    pbar = tqdm(dataloader, desc="   [Валидация]", leave=False, ncols=1000)

    # Отключаем расчет градиентов
    with torch.no_grad():
        # Итерация по батчам
        for batch_data in pbar:
            if batch_data is None: continue
            features, targets, feature_lengths, target_lengths = batch_data
            if features is None or targets is None or feature_lengths is None or target_lengths is None: continue

            batch_size = features.size(0)
            if batch_size == 0: continue

            # Перемещаем данные на устройство
            features = features.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            feature_lengths_cpu = feature_lengths.cpu()
            target_lengths_cpu = target_lengths.cpu()

            # Рассчитываем длины для CTC
            input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)

            loss_value = float('inf')
            lev_dist_batch = float('inf')
            decoded_pairs_batch = []

            try:
                # Прямой проход
                logits = model(features) # (T_red, B, C)
                output_length = logits.shape[0]

                if logits.shape[1] != batch_size:
                    print(f"\n!!! ERROR (Val): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip batch.")
                    continue

                # Log_softmax для CTC
                log_probs = F.log_softmax(logits, dim=2)

                # Клампинг длин
                input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)
                target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])

                # Расчет Loss
                valid_target_mask = target_lengths_clamped > 0
                if not torch.all(valid_target_mask):
                    log_probs_valid = log_probs[:, valid_target_mask, :]
                    targets_valid = targets[valid_target_mask, :]
                    input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]
                    target_lengths_valid = target_lengths_clamped[valid_target_mask]
                    loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
                else:
                    loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)

                if torch.isfinite(loss): loss_value = loss.item()
                else: print("\nWarning (Val): NaN/Inf loss.")

                # Декодирование и расчет Levenshtein
                decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx)
                lev_dist_batch, decoded_pairs_batch = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
                if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0

            except Exception as e:
                print(f"\n!!! Error (Val): {e} !!!")
                traceback.print_exc(limit=1)
                loss_value=float('inf'); lev_dist_batch=float('inf')
                decoded_pairs_batch = [("ERROR","ERROR")] * batch_size # Заполняем ошибками

            # Обновление статистики эпохи
            if np.isfinite(loss_value): # Учитываем только конечные значения loss
                running_loss += loss_value * batch_size
            total_lev_dist += lev_dist_batch * batch_size
            total_samples += batch_size

            # Сохраняем несколько примеров декодирования для вывода
            if len(all_decoded_pairs) < 10: # Сохраняем первые 10 пар
                all_decoded_pairs.extend(decoded_pairs_batch[:max(0, 10 - len(all_decoded_pairs))])

            # Обновление прогресс-бара
            pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}')

    pbar.close() # Закрываем прогресс-бар

    # Вычисляем средние значения за эпоху
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')

    return avg_loss, avg_lev, all_decoded_pairs

print("Функции train_epoch и validate_epoch определены.")
print("\n--- Ячейка 10: Определение функций обучения и валидации завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 10: Определение функций обучения и валидации эпохи ---
Функции train_epoch и validate_epoch определены.

--- Ячейка 10: Определение функций обучения и валидации завершено ---
--------------------------------------------------


### Ячейка 11 Интерактивная настройка RMS

In [194]:
# =============================================================================
# Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (Опционально)
# =============================================================================
print(f"--- Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (SR={AUDIO_CONFIG['sample_rate']} Гц) ---")

if not IPYWIDGETS_AVAILABLE:
    print("Виджеты недоступны. Пропустите эту ячейку или установите ipywidgets.")
else:
    # --- Функции для Визуализации ---
    def plot_mel_spectrogram_interactive(y, sr, n_fft, hop_length, n_mels, fmin, fmax, file_id=""):
        """Строит Мел-спектрограмму."""
        try:
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin=fmin, fmax=fmax)
            S_dB = librosa.power_to_db(S, ref=np.max)

            plt.figure(figsize=(12, 4))
            librosa.display.specshow(S_dB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel', fmin=fmin, fmax=fmax)
            plt.colorbar(format='%+2.0f dB')
            plt.title(f'Мел-Спектрограмма ({file_id})\nn_fft={n_fft}, hop={hop_length}, n_mels={n_mels}, fmax={fmax:.0f}Hz')
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Ошибка при построении спектрограммы: {e}")

    def plot_waveform_interactive(y, sr, file_id=""):
        """Строит волновую форму."""
        plt.figure(figsize=(12, 2))
        librosa.display.waveshow(y, sr=sr, alpha=0.7)
        plt.title(f'Волновая форма ({file_id})')
        plt.tight_layout()
        plt.show()

    # --- Виджеты ---
    # Используем значения из финального AUDIO_CONFIG
    default_sr = AUDIO_CONFIG['sample_rate']
    default_hop = AUDIO_CONFIG['hop_length']
    default_nmels = AUDIO_CONFIG['n_mels']
    default_nfft = AUDIO_CONFIG['n_fft']
    default_fmin = AUDIO_CONFIG['fmin']
    default_fmax = AUDIO_CONFIG['fmax']

    # Выбираем несколько файлов для примера
    example_files = train_df['id'].sample(min(5, len(train_df)), random_state=SEED).tolist()

    file_selector = widgets.Dropdown(options=example_files, description='Файл:')
    # Добавим слайдеры для ключевых параметров, чтобы можно было поиграть
    n_fft_slider = widgets.SelectionSlider(options=[32, 64, 128, 256, 512], value=default_nfft, description='n_fft:', continuous_update=False)
    hop_length_slider = widgets.IntSlider(value=default_hop, min=16, max=256, step=16, description='hop_length:', continuous_update=False)
    n_mels_slider = widgets.IntSlider(value=default_nmels, min=8, max=64, step=4, description='n_mels:', continuous_update=False)
    fmax_slider = widgets.IntSlider(value=int(default_fmax), min=1000, max=default_sr//2, step=100, description='fmax (Hz):', continuous_update=False)

    plot_output = widgets.Output() # Место для вывода графиков

    def update_plot_interactive(change):
        """Обновляет графики при изменении виджетов."""
        with plot_output:
            clear_output(wait=True) # Очищаем предыдущий вывод
            file_id = file_selector.value
            n_fft_val = n_fft_slider.value
            hop_val = hop_length_slider.value
            n_mels_val = n_mels_slider.value
            fmax_val = float(fmax_slider.value)
            fmin_val = default_fmin # Оставляем fmin фиксированным

            if not file_id:
                print("Выберите файл.")
                return

            audio_path = EXTRACTED_AUDIO_DIR / file_id
            if not audio_path.is_file():
                print(f"Файл не найден: {audio_path}")
                return

            try:
                y, sr_loaded = librosa.load(audio_path, sr=default_sr) # Загружаем с нужной SR
                if sr_loaded != default_sr:
                     print(f"Предупреждение: SR файла ({sr_loaded}) отличается от целевой ({default_sr}). Выполнено ресэмплирование.")

                # Строим графики
                print(f"Параметры: n_fft={n_fft_val}, hop={hop_val}, n_mels={n_mels_val}, fmax={fmax_val:.0f}Hz")
                plot_waveform_interactive(y, default_sr, file_id)
                plot_mel_spectrogram_interactive(y, default_sr, n_fft_val, hop_val, n_mels_val, fmin_val, fmax_val, file_id)
                # Отображаем аудио для прослушивания
                display(Audio(data=y, rate=default_sr))

            except Exception as e:
                print(f"Ошибка при обработке файла {file_id}: {e}")
                traceback.print_exc(limit=1)

    # Привязываем обработчик к изменению виджетов
    file_selector.observe(update_plot_interactive, names='value')
    n_fft_slider.observe(update_plot_interactive, names='value')
    hop_length_slider.observe(update_plot_interactive, names='value')
    n_mels_slider.observe(update_plot_interactive, names='value')
    fmax_slider.observe(update_plot_interactive, names='value')

    # Отображаем виджеты и запускаем первое обновление
    print("Выберите файл и настройте параметры для визуализации:")
    display(widgets.VBox([file_selector, n_fft_slider, hop_length_slider, n_mels_slider, fmax_slider, plot_output]))
    update_plot_interactive(None) # Первоначальный вызов для отображения

print("\n--- Ячейка 11: Завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (SR=8000 Гц) ---
Выберите файл и настройте параметры для визуализации:



--- Ячейка 11: Завершена ---
--------------------------------------------------


### Ячейка 12: Функция Полного Цикла Обучения

In [195]:
# =============================================================================
# Ячейка 12: Функция для Полного Цикла Обучения и Инференса (run_training_pipeline)
# =============================================================================
print("--- Ячейка 12: Определение функции run_training_pipeline ---")

# Импорты, необходимые внутри функции
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim.lr_scheduler import OneCycleLR
import numpy as np, pandas as pd, time, json, random, traceback
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Union, Tuple, List, Dict, Optional

# --- MLflow Заглушка (если MLflow не установлен) ---
if not MLFLOW_AVAILABLE:
    class DummyRun:
        def __enter__(self): return self
        def __exit__(self, *args): pass
        class DummyRunInfo: info = type('obj', (object,), {'run_id': 'mlflow_disabled'})()
        info = DummyRunInfo()
    class DummyMLflow:
        def set_experiment(self, *args, **kwargs): pass
        def start_run(self, *args, **kwargs): return DummyRun()
        def log_param(self, *args, **kwargs): pass
        def log_params(self, *args, **kwargs): pass
        def log_metric(self, *args, **kwargs): pass
        def log_metrics(self, *args, **kwargs): pass
        def log_artifact(self, *args, **kwargs): pass
        def log_dict(self, *args, **kwargs): pass
        def set_tag(self, *args, **kwargs): pass
        def active_run(self): return None
        def end_run(self, status='FINISHED'): pass
    mlflow = DummyMLflow()
    print("Используется заглушка для MLflow.")
# --- Конец MLflow Заглушки ---

# --- Проверка глобальных зависимостей пайплайна ---
def check_global_dependencies():
    """Проверяет наличие необходимых классов и функций в глобальной области видимости."""
    required = ['MorseDataset', 'MorseRecognizer', 'collate_fn', 'train_epoch', 'validate_epoch',
                'ctc_greedy_decode', 'calculate_levenshtein', 'set_seed']
    missing = [r for r in required if not callable(globals().get(r))]
    if missing: raise NameError(f"Отсутствуют зависимости: {missing}. Убедитесь, что все ячейки выше выполнены.")

# --- Основная функция пайплайна ---
def run_training_pipeline(
    # Конфигурации
    audio_config: Dict, model_config: Dict, train_config: Dict,
    # Данные и пути
    full_train_df: pd.DataFrame, test_df: pd.DataFrame, audio_dir: Path, base_output_dir: Path,
    # Словари и константы
    char_to_index: Dict[str, int], index_to_char: Dict[int, str],
    blank_idx: int, pad_idx: int, num_classes_ctc: int,
    # Управление
    device: torch.device, run_suffix: str, base_filename_suffix: str,
    seed: int = 42, mlflow_experiment_name: str = "Morse_Experiment"
) -> Dict:
    """
    Выполняет полный цикл: подготовка данных, обучение модели, валидация,
    сохранение лучшей модели и параметров, инференс на тестовых данных,
    сохранение submission файла и логирование в MLflow.
    """
    check_global_dependencies() # Проверяем наличие функций и классов

    # Словарь для хранения результатов этого запуска
    run_results = {
        "run_suffix": run_suffix, "status": "pending", "best_val_lev": float('inf'),
        "best_epoch": None, "final_train_loss": float('inf'), "final_val_loss": float('inf'),
        "train_time_min": 0.0, "infer_time_sec": 0.0, "model_path": None,
        "params_path": None, "submission_path": None, "error": None, "mlflow_run_id": None
    }

    print(f"\n{'='*20} Запуск: {run_suffix} {'='*20}")
    print(f"Audio Config: {json.dumps(audio_config)}")
    print(f"Model Config: {json.dumps(model_config)}")
    print(f"Train Config: {json.dumps(train_config)}")

    # Формирование путей для сохранения артефактов
    output_filename_base = f"{base_filename_suffix}{run_suffix}"
    current_model_path = base_output_dir / f"model_{output_filename_base}.pth"
    current_params_path = base_output_dir / f"params_{output_filename_base}.json"
    current_submission_path = base_output_dir / f"submission_greedy_{output_filename_base}.csv"
    run_results.update({
        "model_path": str(current_model_path),
        "params_path": str(current_params_path),
        "submission_path": str(current_submission_path)
    })
    print(f"  Пути вывода: Model={current_model_path.name}, Params={current_params_path.name}, Sub={current_submission_path.name}")

    # Объявление переменных для блока finally
    model = None; criterion = None; optimizer = None; scheduler = None
    full_dataset = None; train_subset = None; val_subset = None
    train_loader = None; val_loader = None
    inference_model_instance = None; infer_test_dataset = None; test_loader_infer = None
    logits_infer = None; features_infer = None
    best_model_local_path_temp = None # Инициализируем здесь

    try:
        # Установка SEED для воспроизводимости
        set_seed(seed)

        # --- 1. Подготовка Данных ---
        print("\n1. Подготовка данных...")
        full_dataset = MorseDataset(
            dataframe=full_train_df, audio_dir=audio_dir, char_to_index=char_to_index,
            audio_config=audio_config,
            model_input_feature_dim=model_config['input_feature_dim'],
            is_train=True
        )
        dataset_size = len(full_dataset)
        if dataset_size == 0: raise ValueError("Обучающий датасет пуст после инициализации!")

        val_split_ratio = train_config['validation_split_ratio']
        val_size = int(np.floor(val_split_ratio * dataset_size))
        train_size = dataset_size - val_size
        if train_size <= 0 or val_size <= 0: raise ValueError(f"Некорректное разделение: Train={train_size}, Val={val_size}")
        generator = torch.Generator().manual_seed(seed)
        train_subset, val_subset = random_split(full_dataset, [train_size, val_size], generator=generator)

        bs = train_config['batch_size']; nw = train_config['num_workers']; pm = (device.type == 'cuda')
        train_loader = DataLoader(train_subset, batch_size=bs, shuffle=True, collate_fn=collate_fn, num_workers=nw, pin_memory=pm, drop_last=True)
        val_loader = DataLoader(val_subset, batch_size=bs*2, shuffle=False, collate_fn=collate_fn, num_workers=nw, pin_memory=pm)
        print(f"  Данные готовы: Train={len(train_subset)} ({len(train_loader)} батчей), Val={len(val_subset)} ({len(val_loader)} батчей).")

        # --- 2. Инициализация Модели, Loss, Optimizer, Scheduler ---
        print("\n2. Инициализация компонентов...")
        model = MorseRecognizer(num_classes_ctc=num_classes_ctc, **model_config).to(device)
        criterion = nn.CTCLoss(blank=blank_idx, reduction='mean', zero_infinity=True).to(device)
        optimizer_name = train_config.get('optimizer', 'AdamW').lower(); lr = train_config['learning_rate']; wd = train_config['weight_decay']
        if optimizer_name == 'adamw': optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        elif optimizer_name == 'adam': optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        else: print(f"Warning: Неизвестный optimizer '{optimizer_name}'. Используется AdamW."); optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        print(f"  Model, Loss, Optimizer ({type(optimizer).__name__}) готовы.")

        total_batches_in_loader = len(train_loader); batches_per_epoch_run = train_config.get("batches_per_epoch", 0)
        steps_per_epoch = total_batches_in_loader
        if batches_per_epoch_run > 0 and batches_per_epoch_run < total_batches_in_loader: steps_per_epoch = batches_per_epoch_run
        if steps_per_epoch <= 0: raise ValueError(f"Steps_per_epoch ({steps_per_epoch}) должен быть > 0!")
        total_steps = train_config['num_epochs'] * steps_per_epoch
        div_f = train_config.get('div_factor', 25.0); final_div_f = train_config.get('final_div_factor', 1e4)
        scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=total_steps, pct_start=0.3, anneal_strategy='cos', div_factor=div_f, final_div_factor=final_div_f)
        print(f"  Scheduler OneCycleLR создан (total_steps={total_steps}).")

        # --- 3. Цикл Обучения с MLflow ---
        print("\n3. Запуск цикла обучения...")
        mlflow.set_experiment(mlflow_experiment_name)
        with mlflow.start_run(run_name=f"run_{run_suffix}") as current_run:
            mlflow_run_id = current_run.info.run_id
            print(f"  MLflow Run ID: {mlflow_run_id}")
            run_results["mlflow_run_id"] = mlflow_run_id
            run_results["status"] = "training"
            # Логирование параметров запуска
            mlflow.log_param("run_suffix", run_suffix)
            mlflow.log_param("base_filename", base_filename_suffix)
            mlflow.log_params({"seed": seed, **train_config})
            mlflow.log_dict(audio_config, "audio_config.json")
            mlflow.log_dict(model_config, "model_config.json")
            mlflow.set_tag("status", "training")

            start_time_train = time.time()
            best_val_lev = float('inf')
            epochs_without_improvement = 0
            best_epoch = None
            # Путь для временного сохранения лучшей модели локально
            best_model_local_path_temp = base_output_dir / f"temp_best_model_{mlflow_run_id}.pth"

            # Основной цикл по эпохам
            for epoch in range(1, train_config['num_epochs'] + 1):
                print(f"\n--- Эпоха {epoch}/{train_config['num_epochs']} ---")
                avg_train_loss, avg_train_lev, avg_epoch_lr = train_epoch(
                    model, train_loader, criterion, optimizer, scheduler, device, epoch,
                    train_config['num_epochs'], index_to_char, blank_idx, pad_idx,
                    train_config['gradient_clip_norm'], batches_per_epoch=batches_per_epoch_run
                )
                avg_val_loss, avg_val_lev, decoded_examples = validate_epoch(
                    model, val_loader, criterion, device, index_to_char, blank_idx, pad_idx
                )

                run_results.update({ "final_train_loss": avg_train_loss, "final_val_loss": avg_val_loss })
                mlflow.log_metrics({
                    "train_loss": avg_train_loss, "train_levenshtein": avg_train_lev,
                    "val_loss": avg_val_loss, "val_levenshtein": avg_val_lev,
                    "learning_rate": avg_epoch_lr
                }, step=epoch)
                print(f"  Итоги Эпохи {epoch}: Train Loss={avg_train_loss:.4f}, Train Lev={avg_train_lev:.4f}, Val Loss={avg_val_loss:.4f}, Val Lev={avg_val_lev:.4f}")
                print("  Примеры декодирования (Предсказание | Реалность):")
                for pred, real in decoded_examples: print(f"    '{pred}' | '{real}'")

                # Логика сохранения лучшей модели и ранней остановки
                if np.isfinite(avg_val_lev) and avg_val_lev < best_val_lev:
                    print(f"  ✨ Val Lev улучшился: {best_val_lev:.4f} -> {avg_val_lev:.4f}. Сохранение лучшей модели...")
                    best_val_lev = avg_val_lev
                    best_epoch = epoch
                    run_results.update({ "best_val_lev": best_val_lev, "best_epoch": best_epoch })
                    try:
                        # Сохраняем модель локально (временно)
                        torch.save(model.state_dict(), best_model_local_path_temp)
                    except Exception as save_err:
                        print(f"  !!! Ошибка сохранения временной модели: {save_err} !!!")
                    epochs_without_improvement = 0
                else:
                    epochs_without_improvement += 1
                    print(f"  Val Lev не улучшился ({avg_val_lev:.4f} vs best {best_val_lev:.4f}). Без улучшений: {epochs_without_improvement}/{train_config['early_stopping_patience']}")

                if epochs_without_improvement >= train_config['early_stopping_patience']:
                    print(f"  ❗️ Ранняя остановка на эпохе {epoch}!")
                    mlflow.set_tag("status", "completed_early_stopping")
                    run_results["status"] = "completed_early_stopping"
                    break
            # --- Конец цикла по эпохам ---

            if run_results["status"] == "training":
                run_results["status"] = "completed"
                mlflow.set_tag("status", "completed")

            train_duration_min = (time.time() - start_time_train) / 60
            run_results["train_time_min"] = train_duration_min
            print(f"\n  Обучение завершено ({run_results['status']}) за {train_duration_min:.2f} мин. Лучший Val Lev: {best_val_lev:.4f} (Эпоха {best_epoch})")

            # --- 4. Сохранение Лучшей Модели и Параметров ---
            print("\n4. Сохранение артефактов...")
            if np.isfinite(best_val_lev): mlflow.log_metric("best_val_levenshtein", best_val_lev)
            if best_epoch is not None: mlflow.log_metric("best_epoch", best_epoch)
            mlflow.log_metric("training_time_min", train_duration_min)

            model_saved_final = False
            if best_model_local_path_temp is not None and best_model_local_path_temp.exists() and np.isfinite(best_val_lev):
                try:
                    current_model_path.parent.mkdir(parents=True, exist_ok=True)
                    current_model_path.unlink(missing_ok=True)
                    best_model_local_path_temp.rename(current_model_path)
                    print(f"  Лучшая модель сохранена: {current_model_path.name}")
                    mlflow.log_artifact(str(current_model_path), artifact_path="model")
                    model_saved_final = True
                except Exception as e:
                    print(f"  !!! Ошибка перемещения/логирования модели: {e} !!!")
                    if best_model_local_path_temp.exists(): best_model_local_path_temp.unlink()
            else:
                print("  !!! Лучшая модель не найдена или невалидна. Сохранение пропущено.")
                mlflow.set_tag("model_saved", "False")
                if best_model_local_path_temp is not None and best_model_local_path_temp.exists(): best_model_local_path_temp.unlink()

            # Сохранение финальных параметров запуска в JSON
            final_params = {
                'audio_config': audio_config, 'model_config': model_config, 'train_config': train_config,
                'char_map': { 'char_to_index': char_to_index, 'index_to_char': {str(k): v for k, v in index_to_char.items()},
                              'BLANK_IDX': blank_idx, 'PAD_IDX': pad_idx, 'NUM_CLASSES_CTC': num_classes_ctc },
                'results': { k: v for k, v in run_results.items() if k not in ['model_path', 'params_path', 'submission_path', 'mlflow_run_id'] and
                             (np.isfinite(v) if isinstance(v, (float, int)) else v is not None) }
            }
            try:
                current_params_path.parent.mkdir(parents=True, exist_ok=True)
                current_params_path.unlink(missing_ok=True)
                with open(current_params_path, 'w', encoding='utf-8') as f:
                    json.dump(final_params, f, indent=4, ensure_ascii=False)
                print(f"  Параметры сохранены: {current_params_path.name}")
                mlflow.log_artifact(str(current_params_path), artifact_path="config")
            except Exception as e: print(f"  !!! Ошибка сохранения/логирования параметров: {e} !!!")
        # --- Конец MLflow run ---

        # --- 5. Инференс на Тестовых Данных ---
        print("\n5. Запуск инференса...")
        infer_duration_sec = 0.0
        if model_saved_final:
            infer_start_time = time.time()
            try:
                 print(f"  Загрузка артефактов для инференса: {current_model_path.name}, {current_params_path.name}")
                 with open(current_params_path, 'r', encoding='utf-8') as f: loaded_params_inf = json.load(f)
                 loaded_audio_config_inf = loaded_params_inf['audio_config']
                 loaded_model_config_inf = loaded_params_inf['model_config']
                 loaded_char_map_inf = loaded_params_inf['char_map']
                 loaded_index_to_char_inf = {int(k): v for k, v in loaded_char_map_inf['index_to_char'].items()}
                 loaded_blank_idx_inf = loaded_char_map_inf['BLANK_IDX']
                 loaded_num_classes_inf = loaded_char_map_inf['NUM_CLASSES_CTC']

                 inference_model_instance = MorseRecognizer(num_classes_ctc=loaded_num_classes_inf, **loaded_model_config_inf).to(device)
                 try: # Пытаемся загрузить только веса
                     inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device, weights_only=True))
                 except (TypeError, AttributeError):
                     print("   Warning: weights_only=True не поддерживается/вызвал ошибку. Загрузка стандартным способом.")
                     inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device))
                 inference_model_instance.eval()
                 print(f"  Модель для инференса загружена и переведена в режим eval().")

                 print(f"  Создание тестового DataLoader...")
                 infer_test_dataset = MorseDataset(
                     dataframe=test_df, audio_dir=audio_dir, char_to_index=loaded_char_map_inf['char_to_index'],
                     audio_config=loaded_audio_config_inf, model_input_feature_dim=loaded_model_config_inf['input_feature_dim'],
                     is_train=False
                 )
                 infer_bs = train_config.get('batch_size', 16) * 4
                 test_loader_infer = DataLoader(infer_test_dataset, batch_size=infer_bs, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=(device.type == 'cuda') )

                 print(f"  Предсказание на {len(test_df)} тестовых примерах...")
                 predictions: Dict[str, str] = {}
                 pbar_infer = tqdm(test_loader_infer, desc="Инференс", leave=False, ncols=1000)
                 with torch.no_grad():
                     for batch_data_infer in pbar_infer:
                         if batch_data_infer is None: continue
                         features_infer, file_ids_infer, _, _ = batch_data_infer
                         if features_infer is None or file_ids_infer is None or len(features_infer) == 0: continue

                         features_infer = features_infer.to(device, non_blocking=True)
                         try:
                             logits_infer = inference_model_instance(features_infer)
                             decoded_batch_infer = ctc_greedy_decode(logits_infer, loaded_index_to_char_inf, loaded_blank_idx_inf)
                             for file_id, pred_text in zip(file_ids_infer, decoded_batch_infer):
                                 predictions[file_id] = pred_text
                         except Exception as e_inf_batch:
                             print(f"\nОшибка инференса батча: {e_inf_batch}")
                             for fid in file_ids_infer: predictions[fid] = "ERROR_INFER"

                 infer_duration_sec = time.time() - infer_start_time
                 run_results["infer_time_sec"] = infer_duration_sec
                 print(f"  Инференс завершен за {infer_duration_sec:.2f} сек. Сделано предсказаний: {len(predictions)}/{len(test_df)}")

                 if predictions:
                     print(f"  Формирование submission файла: {current_submission_path.name}")
                     submission_df = pd.DataFrame({'id': test_df['id']})
                     submission_df['message'] = submission_df['id'].map(predictions).fillna("ERROR_MISSING")
                     current_submission_path.unlink(missing_ok=True)
                     submission_df.to_csv(current_submission_path, index=False)
                     print(f"  Submission файл сохранен.")
                     mlflow.log_artifact(str(current_submission_path), artifact_path="submission")
                 else:
                     print("  Предсказания не сгенерированы. Submission файл не создан.")
                     mlflow.set_tag("submission_generated", "False")

            except Exception as e_infer:
                print(f"  !!! Ошибка на этапе инференса: {e_infer} !!!")
                traceback.print_exc(limit=2)
                run_results["error"] = f"Inference Error: {e_infer}"
                run_results["status"] = "failed_inference"
                try: mlflow.set_tag("status", "failed_inference")
                except: pass
            finally:
                 try: del inference_model_instance
                 except NameError: pass
                 try: del test_loader_infer
                 except NameError: pass
                 try: del infer_test_dataset
                 except NameError: pass
                 if 'logits_infer' in locals(): del logits_infer
                 if 'features_infer' in locals(): del features_infer
                 if torch.cuda.is_available(): torch.cuda.empty_cache()
        else:
            print("  Инференс пропущен (лучшая модель не была сохранена).")
            run_results["status"] += "_no_inference"
            try: mlflow.set_tag("inference_skipped", "True")
            except: pass

    except Exception as e_main:
        print(f"!!! КРИТИЧЕСКАЯ ОШИБКА в пайплайне {run_suffix}: {e_main} !!!")
        traceback.print_exc()
        run_results["error"] = str(e_main)
        run_results["status"] = "failed_critical"
        try:
            mlflow.set_tag("status", "failed_critical")
            mlflow.set_tag("error", str(e_main)[:250])
            if mlflow.active_run(): mlflow.end_run(status='FAILED')
        except: pass

    finally:
        # Финальная очистка ресурсов
        try: del model
        except NameError: pass
        try: del criterion
        except NameError: pass
        try: del optimizer
        except NameError: pass
        try: del scheduler
        except NameError: pass
        if 'full_dataset' in locals(): del full_dataset
        if 'train_subset' in locals(): del train_subset
        if 'val_subset' in locals(): del val_subset
        if 'train_loader' in locals(): del train_loader
        if 'val_loader' in locals(): del val_loader
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        # Удаляем временный файл модели, если он остался
        if best_model_local_path_temp is not None and best_model_local_path_temp.exists():
             best_model_local_path_temp.unlink()
        print(f"{'='*20} Завершение: {run_suffix} {'='*20}\n")

    return run_results

print("--- Функция run_training_pipeline определена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 12: Определение функции run_training_pipeline ---
--- Функция run_training_pipeline определена ---
--------------------------------------------------


### Ячейка 13: Основной Цикл Обучения

In [ ]:
# =============================================================================
# Ячейка 13: Финальный Запуск (Grouped Conv g=2, Delta win=3)
# =============================================================================
print(f"--- Ячейка 13: Финальный Запуск (Grouped Conv g=2, Delta win=3) ---")

# Импорты и проверки зависимостей
import time, pandas as pd, numpy as np, json, random, copy, traceback
from pathlib import Path; import torch
from IPython.display import display

# Проверка наличия финальных конфигураций и шаблона
required_globals_loop = [
    'train_df', 'test_df', 'EXTRACTED_AUDIO_DIR',
    'char_to_index', 'index_to_char', 'BLANK_IDX', 'PAD_IDX', 'NUM_CLASSES_CTC',
    'device', 'OUTPUT_DIR',
    'BASE_FILENAME_SUFFIX_FINAL', # <-- Используем финальный шаблон
    'SEED', 'run_training_pipeline',
    'AUDIO_CONFIG',
    'MODEL_CONFIG_FINAL', # <-- Используем финальный конфиг модели
    'TRAIN_CONFIG_FINAL',
    'mlflow'
]
missing_globals_loop = [v for v in required_globals_loop if v not in globals() or globals().get(v) is None]
if missing_globals_loop: raise NameError(f"Отсутствуют переменные/функции или они None: {missing_globals_loop}")

# ============================================================================
# === БЛОК 1: Определение Конфигурации для Запуска ===
# ============================================================================
print("\n1. Определение конфигурации для финального запуска...")

config_name = f"Final_MelSpec_DeltaW3_GroupedG2_CNN128_RNN2" # Имя для MLflow и логов
current_model_config = MODEL_CONFIG_FINAL.copy()
current_audio_config = AUDIO_CONFIG.copy()
fixed_train_config = TRAIN_CONFIG_FINAL.copy()

print(f"  Запускаемая конфигурация: '{config_name}'")
print(f"  Аудио параметры: {json.dumps(current_audio_config, indent=2)}")
print(f"  Параметры модели: {json.dumps(current_model_config, indent=2)}")
print(f"  Параметры обучения: {json.dumps(fixed_train_config, indent=2)}")

# --- Инициализация Списка для Результатов ---
all_run_results_list: List[Dict] = []

# ============================================================================
# === БЛОК 2: Запуск обучения для одной конфигурации ===
# ============================================================================
print(f"\n--- Начало финального запуска ({config_name}) ---")
overall_start_time = time.time()
# === Имя эксперимента MLflow ===
MLFLOW_EXPERIMENT_NAME = f"Morse_FinalModel" # Имя для финального эксперимента
# ==============================
print(f"MLflow эксперимент: '{MLFLOW_EXPERIMENT_NAME}'")

# --- Надежное завершение ЛЮБОГО активного MLflow run ---
print("\n--- Попытка завершить ЛЮБОЙ активный MLflow run перед запуском ---")
try:
    if mlflow.active_run():
        active_run_id = mlflow.active_run().info.run_id
        print(f"!!! Обнаружен активный MLflow run ({active_run_id}). Принудительно завершаем... !!!")
        mlflow.end_run()
        time.sleep(1)
        print("--- Проверка: Активный MLflow run отсутствует." if not mlflow.active_run() else "!!! ПРЕДУПРЕЖДЕНИЕ: MLflow run ВСЕ ЕЩЕ АКТИВЕН! !!!")
    else:
        print("--- Активный MLflow run не обнаружен. ---")
except Exception as e_check_run:
    print(f"Предупреждение: Ошибка при проверке/завершении активного MLflow run: {e_check_run}")
# -------------------------------------------------------------

# --- Формируем суффикс для MLflow run ---
current_run_suffix = f"_{config_name}"

# --- Вызов основной функции обучения/инференса ---
try:
    print(f"--- Вызов run_training_pipeline для конфигурации '{config_name}' ---")
    run_result_dict = run_training_pipeline(
        audio_config=current_audio_config,
        model_config=current_model_config,
        train_config=fixed_train_config,
        full_train_df=train_df,
        test_df=test_df,
        audio_dir=EXTRACTED_AUDIO_DIR,
        base_output_dir=OUTPUT_DIR, # Используем финальную папку вывода
        char_to_index=char_to_index,
        index_to_char=index_to_char,
        blank_idx=BLANK_IDX,
        pad_idx=PAD_IDX,
        num_classes_ctc=NUM_CLASSES_CTC,
        device=device,
        run_suffix=current_run_suffix,
        base_filename_suffix=BASE_FILENAME_SUFFIX_FINAL, # <-- Используем финальный шаблон
        seed=SEED,
        mlflow_experiment_name=MLFLOW_EXPERIMENT_NAME
    )
    print(f"--- Успешное завершение run_training_pipeline для '{config_name}' ---")

except Exception as e_pipeline:
    print(f"!!! КРИТИЧЕСКАЯ ОШИБКА ПРИ ВЫЗОВЕ ПАЙПЛАЙНА для '{config_name}' !!!")
    print(f"Ошибка: {e_pipeline}")
    traceback.print_exc()
    run_result_dict = {
        "run_suffix": current_run_suffix,
        "status": "failed_launch",
        "best_val_lev": float('inf'),
        "error": f"Pipeline launch error: {e_pipeline}",
        "config_name": config_name
    }

# Добавляем параметры этого запуска в результаты
run_result_dict['config_name'] = config_name
run_result_dict['n_mels'] = current_audio_config['n_mels']
run_result_dict['n_fft'] = current_audio_config['n_fft']
run_result_dict['hop_length'] = current_audio_config['hop_length']
run_result_dict['feature_type'] = current_audio_config['feature_type']
run_result_dict['delta_win_length'] = current_audio_config.get('delta_win_length', 'N/A')
run_result_dict['num_feature_groups'] = current_model_config.get('num_feature_groups', 'N/A')
all_run_results_list.append(run_result_dict)

# --- Конец запуска ---
# ============================================================================
# === КОНЕЦ БЛОКА 2 ===
# ============================================================================

overall_end_time = time.time()
total_duration_hours = (overall_end_time - overall_start_time) / 3600
print(f"\n--- Финальный запуск ({config_name}) завершен за {total_duration_hours:.2f} часов ---")

# ============================================================================
# === БЛОК 3: Анализ и вывод итогов ===
# ============================================================================
print(f"\n--- Итоги Финального Запуска ({config_name}) ---")
if not all_run_results_list:
    print("Нет результатов для анализа.")
else:
    results_df = pd.DataFrame(all_run_results_list)

    # Настройки отображения Pandas
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_columns', 20)
    pd.set_option('display.width', 180)
    pd.set_option('display.float_format', '{:.4f}'.format)

    print("\nСводная таблица результатов:")
    display_cols = ['config_name', 'best_val_lev', 'best_epoch', 'status', 'train_time_min', 'infer_time_sec', 'error', 'mlflow_run_id']
    display_cols_present = [col for col in display_cols if col in results_df.columns]
    display(results_df[display_cols_present].head(len(results_df)))

    if not results_df.empty:
        best_run = results_df.iloc[0] # Результат единственного запуска
        if pd.notna(best_run['best_val_lev']) and np.isfinite(best_run['best_val_lev']):
            print(f"\n--- Результат Финального Запуска ---")
            print(f"  Конфигурация: {best_run.get('config_name', 'N/A')}")
            print(f"  Best Val Levenshtein: {best_run['best_val_lev']:.4f} (Эпоха {best_run.get('best_epoch', 'N/A')})")
            print(f"  Статус: {best_run.get('status', 'N/A')}")
            if pd.notna(best_run.get('error')):
                 print(f"  Ошибка: {best_run['error']}")
            # Вывод путей к артефактам
            print(f"  Модель сохранена в: {best_run.get('model_path', 'N/A')}")
            print(f"  Параметры сохранены в: {best_run.get('params_path', 'N/A')}")
            print(f"  Submission сохранен в: {best_run.get('submission_path', 'N/A')}")
            print(f"  MLflow Run ID: {best_run.get('mlflow_run_id', 'N/A')}")
        else:
            print("\nНе удалось найти валидный результат (best_val_lev is NaN or Inf).")
            if pd.notna(best_run.get('error')): print(f"  Ошибка: {best_run['error']}")
    else:
        print("\nНе удалось определить результат (DataFrame пуст).")

    # Сохранение полной таблицы результатов (для одного запуска)
    results_csv_path = OUTPUT_DIR / f"final_run_results_{config_name}.csv"
    try:
        results_df.to_csv(results_csv_path, index=False)
        print(f"\nПолные результаты сохранены в CSV: {results_csv_path}")
    except Exception as e:
        print(f"\nНе удалось сохранить результаты в CSV: {e}")
# ============================================================================
# === КОНЕЦ БЛОКА 3 ===
# ============================================================================

print(f"\n--- Ячейка 13: Завершена (Финальный Запуск) ---")
print("-" * 50)
# =============================================================================

--- Ячейка 13: Финальный Запуск (Grouped Conv g=2, Delta win=3) ---

1. Определение конфигурации для финального запуска...
  Запускаемая конфигурация: 'Final_MelSpec_DeltaW3_GroupedG2_CNN128_RNN2'
  Аудио параметры: {
  "feature_type": "melspec_delta_win3_grouped2",
  "sample_rate": 8000,
  "n_fft": 128,
  "hop_length": 64,
  "n_mels": 12,
  "fmin": 0.0,
  "fmax": 4200.0,
  "power": 2.0,
  "apply_trimming": false,
  "trim_top_db": 30,
  "delta_win_length": 3
}
  Параметры модели: {
  "input_feature_dim": 24,
  "cnn_out_channels": [
    128,
    128,
    128
  ],
  "cnn_kernel_size": 9,
  "cnn_stride": 1,
  "cnn_padding": "same",
  "cnn_pool_kernel": 2,
  "rnn_hidden_size": 512,
  "rnn_num_layers": 3,
  "dropout_rate": 0.2,
  "activation_fn": "GELU",
  "classifier_type": "single",
  "num_feature_groups": 2
}
  Параметры обучения: {
  "batch_size": 8,
  "num_workers": 0,
  "num_epochs": 80,
  "learning_rate": 0.0001,
  "div_factor": 10.0,
  "final_div_factor": 10000,
  "weight_decay": 0.

Эпоха 1/80 [Тренировка]:   0%|                                                                                …

   [Валидация]:   0%|                                                                                         …

  Итоги Эпохи 1: Train Loss=4.5219, Train Lev=8.9589, Val Loss=4.1561, Val Lev=8.8780
  Примеры декодирования (Предсказание | Реалность):
    '' | 'ФААР834ОМП'
    '' | 'Т0С ЖЩ0О БМЫ'
    '' | 'ЬЛННЖЗУЯН3ЭХ7'
    '' | 'АИИ9Ъ9ОЭЯЕ'
    '' | '#6ЧФ8ЮО'
    '' | 'ЫЭ# НЬЦЦ'
    '' | '8ЖЭК ЛТЕ'
    '' | '5О4ГЪЦ'
    '' | 'ДФЯЯХЭФ9'
    '' | 'ЧШ9А 8П'
  ✨ Val Lev улучшился: inf -> 8.8780. Сохранение лучшей модели...

--- Эпоха 2/80 ---


Эпоха 2/80 [Тренировка]:   0%|                                                                                …

...

  Итоги Эпохи 36: Train Loss=0.0832, Train Lev=0.2347, Val Loss=0.3215, Val Lev=0.6360
  Примеры декодирования (Предсказание | Реалность):
    'ФААР83ЮОМП' | 'ФААР834ОМП'
    'ТТОС 5Щ0З БМЫ' | 'Т0С ЖЩ0О БМЫ'
    'ЬЛННЖЗУЯН3ЭХ7' | 'ЬЛННЖЗУЯН3ЭХ7'
    'АИИ9Ъ9ОЭЯЕ' | 'АИИ9Ъ9ОЭЯЕ'
    '#6ЧФ8ЮО' | '#6ЧФ8ЮО'
    'ЫЭ# НЬЦЦ' | 'ЫЭ# НЬЦЦ'
    '8ЖЭК ЛТЕ' | '8ЖЭК ЛТЕ'
    '5О4ГЪЦ' | '5О4ГЪЦ'
    'РЯЯХЗФ9' | 'ДФЯЯХЭФ9'
    'ЧШ9А 8П' | 'ЧШ9А 8П'
  Val Lev не улучшился (0.6360 vs best 0.6077). Без улучшений: 6/12

--- Эпоха 37/80 ---


Эпоха 37/80 [Тренировка]:   0%|                                                                               …

   [Валидация]:   0%|                                                                                         …

  Итоги Эпохи 37: Train Loss=0.0769, Train Lev=0.2189, Val Loss=0.3175, Val Lev=0.6273
  Примеры декодирования (Предсказание | Реалность):
    'ФААР83ЮОМП' | 'ФААР834ОМП'
    'ТТОС 5Щ0З БМЫ' | 'Т0С ЖЩ0О БМЫ'
    'ЬЛННЖЗУЯН3ЭХ7' | 'ЬЛННЖЗУЯН3ЭХ7'
    'АИИ9Ъ9ОЭЯЕ' | 'АИИ9Ъ9ОЭЯЕ'
    '#6ЧФ8ЮО' | '#6ЧФ8ЮО'
    'ЫЭ# НЬЦЦ' | 'ЫЭ# НЬЦЦ'
    '8ЖЭК ЛТЕ' | '8ЖЭК ЛТЕ'
    '5О4ГЪЦ' | '5О4ГЪЦ'
    'РРЯЯХ7Ф9' | 'ДФЯЯХЭФ9'
    'ЧШ9А 8П' | 'ЧШ9А 8П'
  Val Lev не улучшился (0.6273 vs best 0.6077). Без улучшений: 7/12

--- Эпоха 38/80 ---


Эпоха 38/80 [Тренировка]:   0%|                                                                               …